In [52]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial


######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))



################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'musicDBArtistMap'

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
retval = dbAP.artist.getData('/Volumes/Piggy/Discog/artists-deezer/2/89128202.p')
retval.show()

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
dbAP.parse(2, previousDays=1, force=True)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [7]:
import urllib
def downloadURL(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    print("Now Downloading {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
        
    return data, response.getcode()

#data, code = downloadURL("https://www.deezer.com/us/artist/5395257/top_track")

In [8]:
url="https://www.iheart.com/artist/helgi-bjornsson-32568452/albums"
data, code = downloadURL(url)
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="iheart_albums.p")
from time import sleep
sleep(3)
url="https://www.iheart.com/artist/helgi-bjornsson-32568452/songs"
data, code = downloadURL(url)
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="iheart_songs.p")

Now Downloading https://www.iheart.com/artist/helgi-bjornsson-32568452/albums
  --> This file is 18.0kB.
Now Downloading https://www.iheart.com/artist/helgi-bjornsson-32568452/songs
  --> This file is 11.4kB.


In [9]:
from webUtils import getHTML
bsdata = getHTML("iheart_test.p")

In [50]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass
from strUtils import fixName
import json
from dbUtils import utilsIHeart
from hashlib import md5


class artistIHeart(artistDBBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        self.dutils = utilsIHeart()
        self.debug  = False
        
        
    ##############################################################################################################################
    ## Parse Data
    ##############################################################################################################################
    def getData(self, inputdata):
        self.getDataBase(inputdata)
        self.checkData()
        
        artist      = self.getName()
        meta        = self.getMeta()
        url         = self.getURL(meta)
        ID          = self.getID(url.url)
        pages       = self.getPages()
        profile     = self.getProfile()        
        media       = self.getMedia(artist)
        mediaCounts = self.getMediaCounts(media)
        
        err = [artist.err, meta.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc
    
    

    ##############################################################################################################################
    ## Artist Name
    ##############################################################################################################################
    def getName(self):
        script = self.bsdata.find("script", {"data-name": "initial-state"})
        if script is None:
            anc = artistDBNameClass(name=None, err = "NoJSON")
            return amc
        
        try:
            jdata = json.loads(script.contents[0])
        except:
            anc = artistDBNameClass(name=None, err = "BadJSON")
            return anc
        

        artistName = None
        for artistID, artistData in jdata['artists']['artists'].items():
            artistName = artistData["artistName"]
            break

        if artistName is not None:
            anc = artistDBNameClass(name=artistName, err=None)
            return anc

        anc = artistDBNameClass(name=None, err = "NoArtistName")
        return anc

    
    

    ##############################################################################################################################
    ## Meta Information
    ##############################################################################################################################
    def getMeta(self):
        metatitle = self.bsdata.find("meta", {"property": "og:title"})
        metaurl   = self.bsdata.find("meta", {"property": "og:url"})

        title = None
        if metatitle is not None:
            title = metatitle.attrs['content']

        url = None
        if metatitle is not None:
            url = metaurl.attrs['content']

        amc = artistDBMetaClass(title=title, url=url)
        return amc
        

    ##############################################################################################################################
    ## Artist URL
    ##############################################################################################################################
    def getURL(self, meta):
        url = meta.url
        auc = artistDBURLClass(url=url)
        return auc
    
    

    ##############################################################################################################################
    ## Artist ID
    ##############################################################################################################################
    def getID(self, url):
        artistID = self.dutils.getArtistID(url, debug=False)
        if artistID is not None:
            aic = artistDBIDClass(ID=artistID)
        else:
            aic = artistDBIDClass(ID=None, err="NoID")
        return aic


    
    ##############################################################################################################################
    ## Artist Pages
    ##############################################################################################################################
    def getPages(self):
        apc   = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        return apc
    
    

    ##############################################################################################################################
    ## Artist Variations
    ##############################################################################################################################
    def getProfile(self):              
        data = {}
        
        artistdiv  = self.bsdata.find("div", {"id": "tlmdata"})
        if artistdiv is not None:
            artistdata = artistdiv.attrs['data-tealium-data']
        else:
            artistdata = None
    
        if artistdata is not None:
            try:
                artistvals = json.loads(artistdata)
                genres     = artistvals["tag"]
            except:
                genres     = None

            if genres is not None:
                genres = genres.split(",")
            else:
                genres = None
        else:
            genres = None
        
       
        data["Profile"] = {'genre': genres, 'style': None}
               
        apc = artistDBProfileClass(profile=data.get("Profile"), aliases=data.get("Aliases"),
                                 members=data.get("Members"), groups=data.get("In Groups"),
                                 sites=data.get("Sites"), variations=data.get("Variations"))
        return apc

    
    
    ##############################################################################################################################
    ## Artist Media
    ############################################################################################################################## 
    def getArtistMediaData(self, jdata):
        media = {"Albums": [], "Singles": []}
        for artistID, artistData in jdata['artists']['artists'].items():
            artistName = artistData["artistName"]
            artistBio  = artistData["artistBio"]
            albums     = artistData["albums"]
            tracks     = artistData["tracks"]

            mediaType = "Albums"
            for albumData in albums:
                albumID   = albumData['albumId']
                albumName = albumData['title']
                albumEpoch = albumData['releaseDate']
                amdc = artistDBMediaDataClass(album=albumName, url=None, aclass=None, aformat=None, artist=[artistName], code=albumID, year=albumEpoch)
                media[mediaType].append(amdc)

            mediaType = "Singles"
            for trackData in tracks:
                trackID   = trackData['trackId']
                trackName = trackData['title']
                amdc = artistDBMediaDataClass(album=trackName, url=None, aclass=None, aformat=None, artist=[artistName], code=trackID, year=None)
                media[mediaType].append(amdc)
        return media
        
    def getTrackMediaData(self, jdata):
        albums = []
        for artistID,artistData in jdata['albums'].items():
            if artistID == "albums":
                continue
            albumsData = artistData["albums"]
            for albumData in albumsData:
                albumID     = albumData["albumId"]
                artistName  = albumData["artistName"]
                albumName   = albumData["title"]
                albumTracks = albumData["tracks"]

                tracks = []
                for track in albumTracks:
                    trackID    = track['id']
                    trackTitle = track['title']
                    trackPos   = track['albumInfo'].get('trackNumber')
                    try:
                        int(trackPos)
                    except:
                        trackPos = -1

                    tracks.append({"ID": trackID, "Title": trackTitle, "Pos": trackPos})
        return None
                
        
        
    def getMedia(self, artist):
        amc  = artistDBMediaClass()
        name = "Albums"
        amc.media[name] = []
        
        script = self.bsdata.find("script", {"data-name": "initial-state"})
        if script is None:
            print("No media data")
            return amc
        
        try:
            jdata = json.loads(script.contents[0])
        except:
            print("Could not load JSON data")
            return amc

        mediaData = self.getArtistMediaData(jdata)
        for mediaType,mediaTypeData in mediaData.items():
            if amc.media.get(mediaType) is None:
                amc.media[mediaType] = []
            for amdc in mediaTypeData:
                amc.media[mediaType].append(amdc)

        return amc
    
    

    ##############################################################################################################################
    ## Artist Media Counts
    ##############################################################################################################################
    def getMediaCounts(self, media):
        amcc = artistDBMediaCountsClass()
        
        credittype = "Releases"
        if amcc.counts.get(credittype) == None:
            amcc.counts[credittype] = {}
        for creditsubtype in media.media.keys():
            amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))
            
        return amcc
        
        
        amcc.err = "No Counts"
        return amcc
        
        results = self.bsdata.findAll("ul", {"class": "facets_nav"})
        if results is None or len(results) == 0:
            amcc.err = "No Counts"
            return amcc
            
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if amcc.counts.get(credittype) == None:
                                amcc.counts[credittype] = {}
                            if amcc.counts[credittype].get(creditsubtype) == None:
                                try:
                                    amcc.counts[credittype][creditsubtype] = int(count)
                                except:
                                    amcc.counts[credittype][creditsubtype] = count
                                    amcc.err = "Non Int"

        return amcc

In [51]:
ai = artistIHeart()
retval = ai.getData("iheart_albums.p")
retval.show()

Could not determine artistID from URL https://www.iheart.com/artist/helgi-bjornsson-32568452/albums/
Artist Data Class
-------------------------
Artist:  Helgi Björnsson
Meta:    Stream Free Music from Albums by Helgi Björnsson | iHeartRadio
         https://www.iheart.com/artist/helgi-bjornsson-32568452/albums/
URL:     https://www.iheart.com/artist/helgi-bjornsson-32568452/albums/
ID:      
Profile: {'profile': {'genre': None, 'style': None}, 'aliases': None, 'members': None, 'sites': None, 'groups': None, 'variations': None, 'search': None, 'external': None, 'err': None}
Pages:   {'ppp': 1, 'tot': 1, 'pages': 1, 'err': None, 'more': False, 'redo': False}
Media:   {'counts': {'Releases': {'Albums': 19, 'Singles': 10}}, 'err': None}
   Albums
      Það bera sig allir vel
      Ég Stoppa Hnöttinn Með Puttanum
      Strax Í Dag
      Einn Af Okkar Allra Bestu Mönnum
      Dönsuðum Á Húsþökum
      Helgi Björns
      Áður Oft Ég Hef
      Ég Fer Á Land Rover Frá Mývatni Á Kópasker
      

In [43]:
from webUtils import getHTML
bsdata = getHTML("iheart_albums.p")

In [49]:
x = 'https://www.iheart.com/artist/helgi-bjornsson-32568452/albums/'
x[:-8]

'https://www.iheart.com/artist/helgi-bjornsson-32568452'

In [44]:
bsdata

<!DOCTYPE html>
<html lang="en" xmlns:fb="http://ogp.me/ns/fb#"><head><title data-react-helmet="true">Stream Free Music from Albums by Helgi Björnsson | iHeartRadio</title><meta content="on" data-react-helmet="true" http-equiv="x-dns-prefetch-control"/><meta charset="utf-8" data-react-helmet="true"/><meta content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no" data-react-helmet="true" name="viewport"/><meta content="yes" data-react-helmet="true" name="mobile-web-app-capable"/><meta content="#C6002B" data-react-helmet="true" name="theme-color"/><meta content="Listen to music you love! Create a custom radio station based on your favorite album by Helgi Björnsson on iHeartRadio!" data-react-helmet="true" name="description"/><meta content="121897277851831" data-react-helmet="true" property="fb:app_id"/><meta content="55354819171" data-react-helmet="true" property="fb:pages"/><meta content="profile" data-react-helmet="true" property="og:type"/><meta content="https

In [15]:
script = bsdata.find("script", {"data-name": "initial-state"})
import json
jdata = json.loads(script.contents[0])

In [26]:
import datetime
for artistID, artistData in jdata['artists']['artists'].items():
    artistName = artistData["artistName"]
    artistBio  = artistData["artistBio"]
    albums     = artistData["albums"]
    tracks     = artistData["tracks"]
    print(artistName)
    print(artistBio)
    
    mediaType = "Albums"
    for albumData in albums:
        albumID   = albumData['albumId']
        albumName = albumData['title']
        albumEpoch = albumData['releaseDate']
        #dt = datetime.datetime.fromtimestamp(albumEpoch)
        print(albumName,albumID,albumEpoch)
        
    mediaType = "Singles"
    for trackData in tracks:
        trackID   = trackData['trackId']
        trackName = trackData['title']
        print(trackName,trackID)

Helgi Björnsson
None
Það bera sig allir vel 97601468 1585094400000
Ég Stoppa Hnöttinn Með Puttanum 66915904 1536883200000
Strax Í Dag 66996578 1536278400000
Einn Af Okkar Allra Bestu Mönnum 66955175 1536019200000
Dönsuðum Á Húsþökum 64451090 1531440000000
Helgi Björns 66990301 1488499200000
Áður Oft Ég Hef 66951969 1487116800000
Ég Fer Á Land Rover Frá Mývatni Á Kópasker 66951353 1487116800000
Viltu Dansa? 64439077 1448323200000
Veröldin Er Ný 64431363 1447200000000
Yfir Esjuna 64439387 1447200000000
Ríðum Sem Fjandinn 66949634 1446076800000
Kampavín 66953533 1445644800000
...Syngur Íslenskar Dægurperlur Ásamt Gestum 66961302 1445385600000
Ég Vil Fara Upp Í Sveit 66952611 1445385600000
Þú Komst Í Hlaðið 66950692 1445385600000
Heim Í Heiðardalinn 64444640 1445299200000
Helgi Syngur Hauk 66953724 1444694400000
Eru Ekki Allir Sexy? 64429148 1443657600000
Það bera sig allir vel 97601469
Dönsuðum Á Húsþökum 64451091
Vængir 66915907
Ástin Sefar 66915910
Ég Stoppa Hnöttinn Með Puttanum 669159

In [38]:
for artistID,artistData in jdata['albums'].items():
    if artistID == "albums":
        continue
    print(artistID)
    albumsData = artistData["albums"]
    for albumData in albumsData:
        albumID     = albumData["albumId"]
        artistName  = albumData["artistName"]
        albumName   = albumData["title"]
        albumTracks = albumData["tracks"]
        
        tracks = []
        for track in albumTracks:
            trackID    = track['id']
            trackTitle = track['title']
            trackPos   = track['albumInfo'].get('trackNumber')
            try:
                int(trackPos)
            except:
                trackPos = -1
            
            tracks.append({"ID": trackID, "Title": trackTitle, "Pos": trackPos})

        print(albumName,'\t',len(tracks))



32568452
Það bera sig allir vel 	 1
Ég Stoppa Hnöttinn Með Puttanum 	 8
Strax Í Dag 	 1
Einn Af Okkar Allra Bestu Mönnum 	 1
Dönsuðum Á Húsþökum 	 1
Helgi Björns 	 12
Áður Oft Ég Hef 	 1
Ég Fer Á Land Rover Frá Mývatni Á Kópasker 	 1
Viltu Dansa? 	 1
Veröldin Er Ný 	 10
Yfir Esjuna 	 12
Ríðum Sem Fjandinn 	 14
Kampavín 	 11
...Syngur Íslenskar Dægurperlur Ásamt Gestum 	 21
Ég Vil Fara Upp Í Sveit 	 12
Þú Komst Í Hlaðið 	 12
Heim Í Heiðardalinn 	 12
Helgi Syngur Hauk 	 12
Eru Ekki Allir Sexy? 	 63


In [39]:
from webUtils import getHTML
bsdata = getHTML("iheart_songs.p")

In [40]:
script = bsdata.find("script", {"data-name": "initial-state"})
import json
jdata = json.loads(script.contents[0])

In [42]:
jdata

{'ads': {'adDivRefs': {'adsDivRef': {'current': None},
   'adswizzDivRef': {'current': None},
   'tritonAdDivRef': {'current': None}},
  'adswizz': {'companionZones': 45287,
   'subdomain': 'clearchannel',
   'zoneId': 45286},
  'amazon': {'pubId': '3901',
   'script': '//c.amazon-adsystem.com/aax2/apstag.js'},
  'customAds': {'companion': '',
   'enableCustomAds': True,
   'partnerIds': 'https://yield-op-idsync.live.streamtheworld.com/partnerIds',
   'playing': False,
   'tritonPartnerIds': {},
   'type': 'Adswizz',
   'url': 'https://customradioadloader.preprod01.streamtheworld.net/api/ads/delivery',
   'tritonScript': 'https://playerservices.live.streamtheworld.com/api/idsync.js?stationId=339593&withCredentials=true'},
  'env': 'prod',
  'googleTag': {'dfpInstanceId': '6663',
   'isEnabled': False,
   'isInitialized': False},
  'liveAdShown': False,
  'lotame': {'clientId': 4086,
   'enabled': False,
   'networkId': 4085,
   'thirdPartyId': 'CLCH',
   'threshold': 100},
  'moat': {'

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
dbAP = dbArtistsDeezer()
dbAP.searchForArtist("Gilli", force=False,maxArtists=1,debug=True)

In [ ]:
from musicDBMap import musicDBMap
mdbmap = musicDBMap("Music", init=False, copy=False)

In [ ]:
from time import sleep
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if i < 19:
        continue
    dbAP = dbArtistsDeezer()
    print("i = {0}".format(i))
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(3.5)

In [3]:
mdbmap = musicDBMap("Spotify")

  Loaded 1419 previously matched entries


In [4]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsLastFM import dbArtistsLastFM

from time import sleep
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if i < 551:
        continue
    print("="*50,artistName,"="*50)
    print("i = {0}".format(i))
    dbAP = dbArtistsLastFM()
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(1.5)
    
    dbAP = dbArtistsDeezer()
    dbAP.searchForArtist(artistName,force=False,maxArtists=1,debug=True)
    sleep(1.5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
================================================== London On Da Track ==================================================
i = 551
=========================   lastfm   =========================


===================== Searching For London On Da Track =====================
1 / 30 	: 76625601557 	 London On Da Track 	 /music/London+On+Da+Track
=========================   deezer   =========================


===================== Searching For London On Da Track =====================
1 / 8 	: 8338688 	 London on Da Track 	 /Volumes/Piggy/Discog/artists-deezer/88/8338688.p
Saving /Volumes/Piggy/Discog/artists-deezer/88/8338688.p (force=False)
  --> This file is 83.2kB.
Done. Sleeping for 2 seconds
================================================== CNCO ==================================================
i = 552
=========================   lastfm   =========================


===================== Searching

1 / 29 	: 25840447195 	 Bobby 'Boris' Pickett 	 /music/Bobby+%27Boris%27+Pickett
Saving /Volumes/Piggy/Discog/artists-lastfm/95/25840447195.p (force=False)
  --> This file is 26.3kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Bobby "Boris" Pickett =====================
1 / 20 	: 147949 	 Bobby "Boris" Pickett 	 /Volumes/Piggy/Discog/artists-deezer/49/147949.p
Saving /Volumes/Piggy/Discog/artists-deezer/49/147949.p (force=False)
  --> This file is 116.3kB.
Done. Sleeping for 2 seconds
================================================== 22 Savage ==================================================
i = 563
=========================   lastfm   =========================


===================== Searching For 22 Savage =====================
1 / 30 	: 79226488363 	 22 Savage 	 /music/22+Savage
Saving /Volumes/Piggy/Discog/artists-lastfm/63/79226488363.p (force=False)
  --> This file is 28.5kB.
Done. Sleeping f

=========================   deezer   =========================


===================== Searching For John Lennon =====================
1 / 20 	: 226 	 John Lennon 	 /Volumes/Piggy/Discog/artists-deezer/26/226.p
================================================== The Beach Boys ==================================================
i = 575
=========================   lastfm   =========================


===================== Searching For The Beach Boys =====================
1 / 28 	: 82198451895 	 The Beach Boys 	 /music/The+Beach+Boys
=========================   deezer   =========================


===================== Searching For The Beach Boys =====================
1 / 20 	: 2179 	 The Beach Boys 	 /Volumes/Piggy/Discog/artists-deezer/79/2179.p
================================================== Gene Autry ==================================================
i = 576
=========================   lastfm   =========================


===================== Searching For Gene Autry ===========

================================================== Ziv Zaifman ==================================================
i = 586
=========================   lastfm   =========================


===================== Searching For Ziv Zaifman =====================
1 / 30 	: 8780086323 	 Ziv Zaifman 	 /music/Ziv+Zaifman
Saving /Volumes/Piggy/Discog/artists-lastfm/23/8780086323.p (force=False)
  --> This file is 27.9kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Ziv Zaifman =====================
1 / 1 	: 13651521 	 Ziv Zaifman 	 /Volumes/Piggy/Discog/artists-deezer/21/13651521.p
Saving /Volumes/Piggy/Discog/artists-deezer/21/13651521.p (force=False)
  --> This file is 28.7kB.
Done. Sleeping for 2 seconds
================================================== Natti Natasha ==================================================
i = 587
=========================   lastfm   =========================


====================

Saving /Volumes/Piggy/Discog/artists-deezer/86/8780586.p (force=False)
  --> This file is 26.8kB.
Done. Sleeping for 2 seconds
================================================== Janelle Monáe ==================================================
i = 597
=========================   lastfm   =========================


===================== Searching For Janelle Monáe =====================
1 / 30 	: 61475981353 	 Janelle Monáe 	 /music/Janelle+Mon%C3%A1e
Saving /Volumes/Piggy/Discog/artists-lastfm/53/61475981353.p (force=False)
  --> This file is 29.3kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Janelle Monáe =====================
1 / 9 	: 131935 	 Janelle Monáe 	 /Volumes/Piggy/Discog/artists-deezer/35/131935.p
Saving /Volumes/Piggy/Discog/artists-deezer/35/131935.p (force=False)
  --> This file is 75.0kB.
Done. Sleeping for 2 seconds
================================================== Jordan Davis =====

...

1 / 17 	: 95949106517 	 Soa mattrix 	 /music/Soa+mattrix
=========================   deezer   =========================


===================== Searching For Soa Mattrix =====================
1 / 1 	: 100981612 	 Soa mattrix 	 /Volumes/Piggy/Discog/artists-deezer/12/100981612.p
================================================== Mapara A Jazz ==================================================
i = 1380
=========================   lastfm   =========================


===================== Searching For Mapara A Jazz =====================
1 / 24 	: 9081453586 	 Mapara A Jazz 	 /music/Mapara+A+Jazz
Saving /Volumes/Piggy/Discog/artists-lastfm/86/9081453586.p (force=False)
  --> This file is 24.1kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Mapara A Jazz =====================
1 / 1 	: 78499372 	 Mapara A Jazz 	 /Volumes/Piggy/Discog/artists-deezer/72/78499372.p
Saving /Volumes/Piggy/Discog/artists-deezer/7

1 / 30 	: 16093497847 	 Zain SA 	 /music/Zain+SA
Saving /Volumes/Piggy/Discog/artists-lastfm/47/16093497847.p (force=False)
  --> This file is 23.5kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Zain SA =====================
1 / 4 	: 10096298 	 Zain SA 	 /Volumes/Piggy/Discog/artists-deezer/98/10096298.p
Saving /Volumes/Piggy/Discog/artists-deezer/98/10096298.p (force=False)
  --> This file is 40.2kB.
Done. Sleeping for 2 seconds
================================================== Zandimaz ==================================================
i = 1391
=========================   lastfm   =========================


===================== Searching For Zandimaz =====================
1 / 4 	: 19083654739 	 Zandimaz 	 /music/Zandimaz
Saving /Volumes/Piggy/Discog/artists-lastfm/39/19083654739.p (force=False)
  --> This file is 22.0kB.
Done. Sleeping for 2 seconds
=========================   deezer   ==========

1 / 20 	: 391993 	 Alex Zurdo 	 /Volumes/Piggy/Discog/artists-deezer/93/391993.p
Saving /Volumes/Piggy/Discog/artists-deezer/93/391993.p (force=False)
  --> This file is 96.7kB.
Done. Sleeping for 2 seconds
================================================== Sierra ==================================================
i = 1399
=========================   lastfm   =========================


===================== Searching For Sierra =====================
1 / 29 	: 11427042385 	 Sierra 	 /music/Sierra
Saving /Volumes/Piggy/Discog/artists-lastfm/85/11427042385.p (force=False)
  --> This file is 28.4kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Sierra =====================
No data in {'EPISODE_ID': '233273032', 'EPISODE_STATUS': '1', 'AVAILABLE': True, 'SHOW_ID': '1079542', 'SHOW_NAME': 'Relatos De Horror (Historias De Terror)', 'SHOW_ART_MD5': 'f1a5b0f8e9f903a2b8ed4611cd83d7e3', 'SHOW_DESCRIPTION': 'Proye

1 / 30 	: 25700332634 	 Mika 	 /music/Mika
=========================   deezer   =========================


===================== Searching For Mika =====================
1 / 20 	: 6603 	 MIKA 	 /Volumes/Piggy/Discog/artists-deezer/3/6603.p
Saving /Volumes/Piggy/Discog/artists-deezer/3/6603.p (force=False)
  --> This file is 81.5kB.
Done. Sleeping for 2 seconds
================================================== DADONI ==================================================
i = 1408
=========================   lastfm   =========================


===================== Searching For DADONI =====================
1 / 30 	: 80218944236 	 Dadoni 	 /music/Dadoni
Saving /Volumes/Piggy/Discog/artists-lastfm/36/80218944236.p (force=False)
  --> This file is 21.6kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For DADONI =====================
1 / 20 	: 7455976 	 GAMPER & DADONI 	 /Volumes/Piggy/Discog/artists-deezer/76/74

Saving /Volumes/Piggy/Discog/artists-deezer/95/1495.p (force=False)
  --> This file is 131.8kB.
Done. Sleeping for 2 seconds
================================================== Pearce ==================================================
i = 1418
=========================   lastfm   =========================


===================== Searching For Pearce =====================
1 / 30 	: 3710522852 	 Pearce 	 /music/Pearce
Saving /Volumes/Piggy/Discog/artists-lastfm/52/3710522852.p (force=False)
  --> This file is 22.4kB.
Done. Sleeping for 2 seconds
=========================   deezer   =========================


===================== Searching For Pearce =====================
1 / 20 	: 85479352 	 Pearce Joza 	 /Volumes/Piggy/Discog/artists-deezer/52/85479352.p
Saving /Volumes/Piggy/Discog/artists-deezer/52/85479352.p (force=False)
  --> This file is 20.6kB.
Done. Sleeping for 2 seconds


In [ ]:
data, code = downloadURL("https://www.deezer.com/search/Benny%20Jamz")
saveFile(idata=data, ifile="deezer_search.p")

In [ ]:
from ioUtils import saveFile, getFile
saveFile(idata=data, ifile="deezer_test.p")

In [ ]:
from webUtils import getHTML
bsdata = getHTML("deezer_test.p")

In [ ]:
sdata = getHTML("deezer_search.p")

In [ ]:
import json
for script in sdata.findAll("script"):
    if len(script.contents) == 0:
        continue
        
    if script.contents[0].startswith("window.__DZR_APP_STATE__ = "):        
        try:        
            jdata = json.loads(script.contents[0].replace("window.__DZR_APP_STATE__ = ", ""))
            print("HI")
        except:
            continue

In [ ]:
for result in jdata["TOP_RESULT"]:
    artistID   = result["ART_ID"]
    artistName = result["ART_NAME"]
    
    url = "https://www.deezer.com/us/artist/{0}/top_track".format(artistID)
    savename = "{0}_{1}.p".format(artistID, artistName)
    data, code = downloadURL(url)
    saveFile(idata=data, ifile=savename, debug=True)

In [ ]:
%load_ext autoreload
%autoreload
from artistDeezer import artistDeezer
ad = artistDeezer()
retval = ad.getData("/Volumes/Piggy/Discog/artists-deezer/80/6980.p")
retval.show()

In [ ]:
bsdata = getHTML("5395257_Benny Jamz.p")
bsdata

<h1 class="heading-1" id="naboo_artist_name">
<a class="link-gray-light" href="/us/artist/5395257">
<span itemprop="name">Benny Jamz</span>
</a>
</h1>

In [ ]:
for result in jdata["ARTIST"]['data']:    
    artistID   = result["ART_ID"]
    artistName = result["ART_NAME"]

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=1, force=False)
    
for modVal in range(100):
    break
    dbAP = dbArtistsExtra(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    break
    dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit

for modVal in range(65,75):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=100)

In [ ]:
vals="""=========================   discogs   =========================
  ===> Found 7566 previous data for ModVal=0
Found 336 artists to get
Error downloading https://www.discogs.com/artist/1439900-Adrian-Lux?type=Unofficial
Error downloading https://www.discogs.com/artist/2372000-Blasfemia-3?type=Unofficial
Error downloading https://www.discogs.com/artist/4353300-Keith-Jarrett-Gary-Peacock-Jack-DeJohnette?type=Unofficial
Error downloading https://www.discogs.com/artist/493200-Glennis-Grace?type=Unofficial
Error downloading https://www.discogs.com/artist/582500-Zara-2?type=Unofficial
Error downloading https://www.discogs.com/artist/718900-G-Punkt-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7844 previous data for ModVal=1
Found 347 artists to get
Error downloading https://www.discogs.com/artist/137201-SLM?type=Unofficial
Error downloading https://www.discogs.com/artist/212401-Powerman?type=Unofficial
Error downloading https://www.discogs.com/artist/2428401-Horse-Head-2?type=Unofficial
Error downloading https://www.discogs.com/artist/29101-DONS?type=Unofficial
Error downloading https://www.discogs.com/artist/3853201-Svolder?type=Unofficial
Error downloading https://www.discogs.com/artist/4303701-Pera-Joe-Blues-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/5277801-Orquesta-Imperial-Vienesa?type=Unofficial
Error downloading https://www.discogs.com/artist/68501-Colonel-Mite?type=Unofficial
Error downloading https://www.discogs.com/artist/75001-GTR?type=Unofficial
=========================   discogs   =========================
  ===> Found 7601 previous data for ModVal=2
Found 322 artists to get
Error downloading https://www.discogs.com/artist/2396602-The-Jimmy-Johnson-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/2963602-Johnny-Oliver?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/2/unofficial/309502.p (force=False)
  --> This file is 17.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7870 previous data for ModVal=3
Found 362 artists to get
Error downloading https://www.discogs.com/artist/1154403-Tiny-Hat-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/271303-DJ-Harry-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2750003-The-New-Eyes?type=Unofficial
Error downloading https://www.discogs.com/artist/303-Future-Forces-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/356103-The-Executioners-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4760503-Roger-Waters-Eric-Clapton?type=Unofficial
Error downloading https://www.discogs.com/artist/5652703-We-Love-Beats?type=Unofficial
Error downloading https://www.discogs.com/artist/7303-Aftershock?type=Unofficial
Error downloading https://www.discogs.com/artist/830403-RPM-3?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/3/unofficial/2782403.p (force=False)
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/3/unofficial/4985803.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7695 previous data for ModVal=4
Found 314 artists to get
Error downloading https://www.discogs.com/artist/1390704-Marina-Del-Ray?type=Unofficial
Error downloading https://www.discogs.com/artist/14904-Rhythmatic?type=Unofficial
Error downloading https://www.discogs.com/artist/1684304-Bobby-Krlic?type=Unofficial
Error downloading https://www.discogs.com/artist/210204-The-Rush?type=Unofficial
Error downloading https://www.discogs.com/artist/2570204-James-Last-His-Orchestra-And-Singers?type=Unofficial
Error downloading https://www.discogs.com/artist/32904-Paul-Newman?type=Unofficial
Error downloading https://www.discogs.com/artist/4005604-Yuko-Ishibashi?type=Unofficial
Error downloading https://www.discogs.com/artist/4681404-Lawrence-Brownlee?type=Unofficial
Error downloading https://www.discogs.com/artist/730304-Alfred-Scholz?type=Unofficial
Error downloading https://www.discogs.com/artist/872704-Matsuo-Ono?type=Unofficial
Error downloading https://www.discogs.com/artist/922704-Munir-Bashir?type=Unofficial
Error downloading https://www.discogs.com/artist/940904-Cool-Kids?type=Unofficial
Error downloading https://www.discogs.com/artist/3694504-Cloaca-Maxima?type=Unofficial
=========================   discogs   =========================
  ===> Found 7527 previous data for ModVal=5
Found 344 artists to get
Error downloading https://www.discogs.com/artist/186105-Taraf-de-Ha%C3%AFdouks?type=Unofficial
Error downloading https://www.discogs.com/artist/4005605-Rashufel?type=Unofficial
Error downloading https://www.discogs.com/artist/407805-Carla-White?type=Unofficial
Error downloading https://www.discogs.com/artist/490405-WW-III?type=Unofficial
=========================   discogs   =========================
  ===> Found 7558 previous data for ModVal=6
Found 329 artists to get
Error downloading https://www.discogs.com/artist/11006-TJR?type=Unofficial
Error downloading https://www.discogs.com/artist/1122606-Paul-Lavalle?type=Unofficial
Error downloading https://www.discogs.com/artist/173406-DJ-EmBee?type=Unofficial
Error downloading https://www.discogs.com/artist/255106-Deja-Gruv?type=Unofficial
Error downloading https://www.discogs.com/artist/312006-Andy-Fisher?type=Unofficial
Error downloading https://www.discogs.com/artist/3832906-Golden-Sounds-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/538306-The-Saliva-Commandos?type=Unofficial
Error downloading https://www.discogs.com/artist/6505806-National-Skin?type=Unofficial
Error downloading https://www.discogs.com/artist/745606-The-Art-Van-Damme-Quintet?type=Unofficial
Error downloading https://www.discogs.com/artist/964306-Angelyne?type=Unofficial
=========================   discogs   =========================
  ===> Found 7661 previous data for ModVal=7
Found 311 artists to get
Error downloading https://www.discogs.com/artist/1532407-Steve-Hitchcock-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1570807-The-Hi-Liters?type=Unofficial
Error downloading https://www.discogs.com/artist/189507-Virtuoso-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2085607-Raphy-Leavitt-Y-Su-Orquesta-La-Selecta?type=Unofficial
Error downloading https://www.discogs.com/artist/2866207-2nd-Family?type=Unofficial
Error downloading https://www.discogs.com/artist/311107-Chris-Parker-2?type=Unofficial
Error downloading https://www.discogs.com/artist/5422807-Die-Laughing-5?type=Unofficial
Error downloading https://www.discogs.com/artist/611007-El-Pino-and-the-Volunteers?type=Unofficial
Error downloading https://www.discogs.com/artist/623607-David-Maxwell?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/7/unofficial/2063207.p (force=False)
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7711 previous data for ModVal=8
Found 345 artists to get
Error downloading https://www.discogs.com/artist/13808-Boyd-Jarvis?type=Unofficial
Error downloading https://www.discogs.com/artist/15808-Peter-Howell?type=Unofficial
Error downloading https://www.discogs.com/artist/2666808-Seaside-Lovers?type=Unofficial
Error downloading https://www.discogs.com/artist/2884808-Luder?type=Unofficial
Error downloading https://www.discogs.com/artist/337908-Sugar-Jones?type=Unofficial
Error downloading https://www.discogs.com/artist/84408-Robbie?type=Unofficial
=========================   discogs   =========================
  ===> Found 7616 previous data for ModVal=9
Found 365 artists to get
Error downloading https://www.discogs.com/artist/1534909-Catherine-Ribeiro-Alpes?type=Unofficial
Error downloading https://www.discogs.com/artist/17609-Liaisons-D?type=Unofficial
Error downloading https://www.discogs.com/artist/303209-Machito?type=Unofficial
Error downloading https://www.discogs.com/artist/95409-James-Yorkston?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/9/unofficial/1350809.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7641 previous data for ModVal=10
Found 349 artists to get
Error downloading https://www.discogs.com/artist/1112110-Wally-Gonzales?type=Unofficial
Error downloading https://www.discogs.com/artist/114710-Dustin-Zahn?type=Unofficial
Error downloading https://www.discogs.com/artist/1574510-Pogo-11?type=Unofficial
Error downloading https://www.discogs.com/artist/1779910-Ekoplekz?type=Unofficial
Error downloading https://www.discogs.com/artist/1904310-Yukio-Hashi?type=Unofficial
Error downloading https://www.discogs.com/artist/2048710-Ranfold-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/21210-Ali-B?type=Unofficial
Error downloading https://www.discogs.com/artist/2230410-Conjunto-Black-Power?type=Unofficial
Error downloading https://www.discogs.com/artist/225210-Minor-Detail?type=Unofficial
Error downloading https://www.discogs.com/artist/262610-TISM?type=Unofficial
Error downloading https://www.discogs.com/artist/2627610-Unknown-40?type=Unofficial
Error downloading https://www.discogs.com/artist/310-Decoder?type=Unofficial
Error downloading https://www.discogs.com/artist/898410-Stephen-Keyes?type=Unofficial
=========================   discogs   =========================
  ===> Found 7671 previous data for ModVal=11
Found 341 artists to get
Error downloading https://www.discogs.com/artist/261511-Sponge-3?type=Unofficial
Error downloading https://www.discogs.com/artist/278011-The-Skeptix?type=Unofficial
Error downloading https://www.discogs.com/artist/3371411-Set-It-Off-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4704511-Brett-Ellis-2?type=Unofficial
Error downloading https://www.discogs.com/artist/5270211-Tomasz-Sta%C3%85%C2%84ko-W%C3%85%C2%82adys%C3%85%C2%82aw-Sendecki?type=Unofficial
Error downloading https://www.discogs.com/artist/5811811-The-Bachelors-11?type=Unofficial
Error downloading https://www.discogs.com/artist/66611-Mash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/11/unofficial/262411.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7712 previous data for ModVal=12
Found 331 artists to get
Error downloading https://www.discogs.com/artist/1136212-Georgia-Carr?type=Unofficial
Error downloading https://www.discogs.com/artist/1604012-First-Rebirth?type=Unofficial
Error downloading https://www.discogs.com/artist/166812-Michael-Mann?type=Unofficial
Error downloading https://www.discogs.com/artist/1909412-Ranma-Operatic-Troupe?type=Unofficial
Error downloading https://www.discogs.com/artist/2689812-Willburn-Brothers?type=Unofficial
Error downloading https://www.discogs.com/artist/2727912-Les-T%C3%AAtes-Blanches?type=Unofficial
Error downloading https://www.discogs.com/artist/4955912-EAL-Axner?type=Unofficial
Error downloading https://www.discogs.com/artist/5475512-SDP-5?type=Unofficial
Error downloading https://www.discogs.com/artist/84912-Mutter?type=Unofficial
=========================   discogs   =========================
  ===> Found 7776 previous data for ModVal=13
Found 360 artists to get
Error downloading https://www.discogs.com/artist/21913-T-La-Rock?type=Unofficial
Error downloading https://www.discogs.com/artist/2446213-The-Amity-Affliction?type=Unofficial
Error downloading https://www.discogs.com/artist/2639413-ChefSpecial?type=Unofficial
Error downloading https://www.discogs.com/artist/283913-The-Kilimanjaro-Darkjazz-Ensemble?type=Unofficial
Error downloading https://www.discogs.com/artist/3231613-DJ-%D0%90%D0%BD%D0%B6%D0%B5%D0%BB%D0%B0-%D0%A8%D1%83%D0%BB%D1%8C%D0%B6%D0%B5%D0%BD%D0%BA%D0%BE?type=Unofficial
Error downloading https://www.discogs.com/artist/36113-You?type=Unofficial
Error downloading https://www.discogs.com/artist/39713-Edan?type=Unofficial
Error downloading https://www.discogs.com/artist/5041113-Silk-N-Steel?type=Unofficial
Error downloading https://www.discogs.com/artist/70213-Mr-Gasmask?type=Unofficial
Error downloading https://www.discogs.com/artist/862913-Francisco-T%C3%A1rrega?type=Unofficial
=========================   discogs   =========================
  ===> Found 7527 previous data for ModVal=14
Found 300 artists to get
Error downloading https://www.discogs.com/artist/1017114-David-Tattersall?type=Unofficial
Error downloading https://www.discogs.com/artist/1147814-Cities-Service-Band-Of-America?type=Unofficial
Error downloading https://www.discogs.com/artist/115614-Yuka?type=Unofficial
Error downloading https://www.discogs.com/artist/13914-DJ-Trajic?type=Unofficial
Error downloading https://www.discogs.com/artist/346214-Mr-Review?type=Unofficial
Error downloading https://www.discogs.com/artist/6869914-Fastfinger-Smith?type=Unofficial
=========================   discogs   =========================
  ===> Found 7641 previous data for ModVal=15
Found 312 artists to get
Error downloading https://www.discogs.com/artist/166415-HOP?type=Unofficial
Error downloading https://www.discogs.com/artist/380215-DBN?type=Unofficial
Error downloading https://www.discogs.com/artist/3872015-Eddie-Lebron-And-His-Orch?type=Unofficial
Error downloading https://www.discogs.com/artist/39915-Flesh-Field?type=Unofficial
Error downloading https://www.discogs.com/artist/479215-Xalan?type=Unofficial
=========================   discogs   =========================
  ===> Found 7576 previous data for ModVal=16
Found 339 artists to get
Error downloading https://www.discogs.com/artist/176716-Big-B?type=Unofficial
Error downloading https://www.discogs.com/artist/190016-Manuel-Mota?type=Unofficial
Error downloading https://www.discogs.com/artist/19216-Indo?type=Unofficial
Error downloading https://www.discogs.com/artist/216-Club-69?type=Unofficial
Error downloading https://www.discogs.com/artist/355316-Say-When%21?type=Unofficial
Error downloading https://www.discogs.com/artist/4200216-Paul-Page-5?type=Unofficial
Error downloading https://www.discogs.com/artist/46716-Dom-Salvador-e-Aboli%C3%A7ao?type=Unofficial
Error downloading https://www.discogs.com/artist/4954516-Del-Larks?type=Unofficial
Error downloading https://www.discogs.com/artist/6163416-Manekeni?type=Unofficial
Error downloading https://www.discogs.com/artist/655616-Jerome-Moross?type=Unofficial
Error downloading https://www.discogs.com/artist/667216-Peter-Tork?type=Unofficial
Error downloading https://www.discogs.com/artist/695016-Zoogz-Rift?type=Unofficial
=========================   discogs   =========================
  ===> Found 7658 previous data for ModVal=17
Found 331 artists to get
Error downloading https://www.discogs.com/artist/1004517-Truckfighters?type=Unofficial
Error downloading https://www.discogs.com/artist/136017-Ann-Robinson?type=Unofficial
Error downloading https://www.discogs.com/artist/1700517-Madame-Blavatsky?type=Unofficial
Error downloading https://www.discogs.com/artist/2608717-Ralph-Lundsten-And-The-Andromeda-All-Stars?type=Unofficial
Error downloading https://www.discogs.com/artist/2690617-Northlane?type=Unofficial
Error downloading https://www.discogs.com/artist/28817-Brandy?type=Unofficial
Error downloading https://www.discogs.com/artist/330017-Mac-Dre?type=Unofficial
Error downloading https://www.discogs.com/artist/3495917-Doris-Browne?type=Unofficial
Error downloading https://www.discogs.com/artist/374717-Ville-Valo?type=Unofficial
Error downloading https://www.discogs.com/artist/4180617-General-Store?type=Unofficial
Error downloading https://www.discogs.com/artist/430217-Greg-Van-Bueren?type=Unofficial
=========================   discogs   =========================
  ===> Found 7687 previous data for ModVal=18
Found 347 artists to get
Error downloading https://www.discogs.com/artist/119518-Yuksek?type=Unofficial
Error downloading https://www.discogs.com/artist/1320718-Werner-Stiefel?type=Unofficial
Error downloading https://www.discogs.com/artist/1471218-EAST-6?type=Unofficial
Error downloading https://www.discogs.com/artist/165718-Stanley-Foort?type=Unofficial
Error downloading https://www.discogs.com/artist/231818-Santos-3?type=Unofficial
Error downloading https://www.discogs.com/artist/24018-Cyba-Space?type=Unofficial
Error downloading https://www.discogs.com/artist/2519018-FM-Freeks?type=Unofficial
Error downloading https://www.discogs.com/artist/362918-Vic-Rawlings?type=Unofficial
Error downloading https://www.discogs.com/artist/475918-Louis-Neefs?type=Unofficial
Error downloading https://www.discogs.com/artist/4889118-Alt%C3%BA-Pag%C3%A1n%C3%A1ch?type=Unofficial
Error downloading https://www.discogs.com/artist/60718-DJ-Vas?type=Unofficial
=========================   discogs   =========================
  ===> Found 7624 previous data for ModVal=19
Found 356 artists to get
Error downloading https://www.discogs.com/artist/2198919-Antiquus-Scriptum?type=Unofficial
Error downloading https://www.discogs.com/artist/230519-Nicolay?type=Unofficial
Error downloading https://www.discogs.com/artist/2567719-Eden-44?type=Unofficial
Error downloading https://www.discogs.com/artist/3110519-Lloyd-Sims?type=Unofficial
Error downloading https://www.discogs.com/artist/3881519-The-Chorus-And-Orchestra-Of-The-Metropolitan-Opera-Association?type=Unofficial
Error downloading https://www.discogs.com/artist/4934319-Goran-Bregovi%C3%84%C2%87s-Karmen?type=Unofficial
Error downloading https://www.discogs.com/artist/5890419-Rita-Sakura?type=Unofficial
Error downloading https://www.discogs.com/artist/60419-Ruff-With-The-Smooth?type=Unofficial
Error downloading https://www.discogs.com/artist/68219-The-Masterdon-Committee?type=Unofficial
=========================   discogs   =========================
  ===> Found 7678 previous data for ModVal=20
Found 370 artists to get
Error downloading https://www.discogs.com/artist/1210920-Royce-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/1358820-Nick-Curran?type=Unofficial
Error downloading https://www.discogs.com/artist/2120-Capone?type=Unofficial
Error downloading https://www.discogs.com/artist/2151720-John-Lundvik?type=Unofficial
Error downloading https://www.discogs.com/artist/4220-X?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/20/unofficial/4723220.p (force=False)
  --> This file is 21.3kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/49020-Moksha?type=Unofficial
Error downloading https://www.discogs.com/artist/504620-Blood-Of-Christ?type=Unofficial
Error downloading https://www.discogs.com/artist/637620-Henry-Mancini-And-His-Orchestra-And-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/660620-Chuck-Ragan?type=Unofficial
Error downloading https://www.discogs.com/artist/750620-Suze?type=Unofficial
=========================   discogs   =========================
  ===> Found 7627 previous data for ModVal=21
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1215121-The-Rising-Storm?type=Unofficial
Error downloading https://www.discogs.com/artist/1222921-Queen-Paul-Rodgers?type=Unofficial
Error downloading https://www.discogs.com/artist/2207421-Sorcery-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2220821-Most-Dope?type=Unofficial
Error downloading https://www.discogs.com/artist/2709921-IOEarth?type=Unofficial
Error downloading https://www.discogs.com/artist/284821-Leah-Buckareff?type=Unofficial
Error downloading https://www.discogs.com/artist/5076121-The-Soulettes-5?type=Unofficial
Error downloading https://www.discogs.com/artist/728821-Postage?type=Unofficial
Error downloading https://www.discogs.com/artist/74321-Trickster-CMVs?type=Unofficial
Error downloading https://www.discogs.com/artist/801621-Sonic-Reducer?type=Unofficial
=========================   discogs   =========================
  ===> Found 7650 previous data for ModVal=22
Found 360 artists to get
Error downloading https://www.discogs.com/artist/2070422-Ravi-Shankar-Ali-Akbar-Khan?type=Unofficial
Error downloading https://www.discogs.com/artist/2294222-Dopethrone?type=Unofficial
Error downloading https://www.discogs.com/artist/269222-Ahmed-Abdullah?type=Unofficial
Error downloading https://www.discogs.com/artist/305622-Pluton-Svea?type=Unofficial
Error downloading https://www.discogs.com/artist/348622-Mike-Czech?type=Unofficial
Error downloading https://www.discogs.com/artist/3886122-Tony-Key?type=Unofficial
Error downloading https://www.discogs.com/artist/4753422-The-Chateaus-4?type=Unofficial
Error downloading https://www.discogs.com/artist/518522-Crowpath?type=Unofficial
Error downloading https://www.discogs.com/artist/67822-MR?type=Unofficial
Error downloading https://www.discogs.com/artist/755522-Peggy-Seeger?type=Unofficial
=========================   discogs   =========================
  ===> Found 7656 previous data for ModVal=23
Found 335 artists to get
Error downloading https://www.discogs.com/artist/230723-Taxman-2?type=Unofficial
Error downloading https://www.discogs.com/artist/23223-JD-Braithwaite?type=Unofficial
Error downloading https://www.discogs.com/artist/2639123-JJ-Jellybean?type=Unofficial
Error downloading https://www.discogs.com/artist/277223-Anarchist-Academy?type=Unofficial
Error downloading https://www.discogs.com/artist/4563723-Dealey-Plaza?type=Unofficial
Error downloading https://www.discogs.com/artist/586723-Jeremy-Fish?type=Unofficial
=========================   discogs   =========================
  ===> Found 7458 previous data for ModVal=24
Found 302 artists to get
Error downloading https://www.discogs.com/artist/1217124-Salvation-Army-New-York-Staff-Band-And-Male-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/2480624-Violet-Poison?type=Unofficial
Error downloading https://www.discogs.com/artist/306524-Willie-Alexander?type=Unofficial
Error downloading https://www.discogs.com/artist/328224-Rajoitus?type=Unofficial
Error downloading https://www.discogs.com/artist/4124-DJ-Friction?type=Unofficial
Error downloading https://www.discogs.com/artist/44024-Hurricane?type=Unofficial
Error downloading https://www.discogs.com/artist/4628024-The-Void-Pacific-Choir?type=Unofficial
Error downloading https://www.discogs.com/artist/6459024-Saved-My-Life?type=Unofficial
Error downloading https://www.discogs.com/artist/6690124-Guerilla-Tek?type=Unofficial
Error downloading https://www.discogs.com/artist/789424-Carmine-Appice-Project?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/24/unofficial/21724.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7632 previous data for ModVal=25
Found 314 artists to get
Error downloading https://www.discogs.com/artist/10825-Sugizo?type=Unofficial
Error downloading https://www.discogs.com/artist/181225-Nouri?type=Unofficial
Error downloading https://www.discogs.com/artist/252025-Pat-Patrick?type=Unofficial
Error downloading https://www.discogs.com/artist/5258925-Small-Black-Lambs?type=Unofficial
Error downloading https://www.discogs.com/artist/666425-Ryan-Unks?type=Unofficial
Error downloading https://www.discogs.com/artist/92825-The-Warlocks?type=Unofficial
=========================   discogs   =========================
  ===> Found 7723 previous data for ModVal=26
Found 317 artists to get
Error downloading https://www.discogs.com/artist/1277826-Vega-15?type=Unofficial
Error downloading https://www.discogs.com/artist/1574126-Doctor-Stereo?type=Unofficial
Error downloading https://www.discogs.com/artist/252526-The-Daily-Flash?type=Unofficial
Error downloading https://www.discogs.com/artist/288726-Moss-6?type=Unofficial
Error downloading https://www.discogs.com/artist/374226-Plies?type=Unofficial
Error downloading https://www.discogs.com/artist/422626-Sad-2?type=Unofficial
Error downloading https://www.discogs.com/artist/477926-AOK-3?type=Unofficial
Error downloading https://www.discogs.com/artist/48426-20th-Century-Steel-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/55326-Andrew-Chalk?type=Unofficial
Error downloading https://www.discogs.com/artist/629226-The-Longcut?type=Unofficial
Error downloading https://www.discogs.com/artist/67226-Stacey-Pullen?type=Unofficial
Error downloading https://www.discogs.com/artist/88426-Danny-Jay?type=Unofficial
=========================   discogs   =========================
  ===> Found 7546 previous data for ModVal=27
Found 317 artists to get
Saving /Volumes/Piggy/Discog/artists-discogs/27/unofficial/1583727.p (force=False)
  --> This file is 19.6kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/1727-Special-K?type=Unofficial
Error downloading https://www.discogs.com/artist/2056527-The-Hepsters?type=Unofficial
Error downloading https://www.discogs.com/artist/232827-Miss-Jones-2?type=Unofficial
Error downloading https://www.discogs.com/artist/281527-Luciano-Michelini?type=Unofficial
Error downloading https://www.discogs.com/artist/386327-Jim-Pembroke?type=Unofficial
Error downloading https://www.discogs.com/artist/5093227-Zac-Master?type=Unofficial
Error downloading https://www.discogs.com/artist/70427-Speech-Defect?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/27/unofficial/2949127.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7677 previous data for ModVal=28
Found 353 artists to get
Saving /Volumes/Piggy/Discog/artists-discogs/28/unofficial/1249728.p (force=False)
  --> This file is 27.7kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/127628-Mark-Robinson?type=Unofficial
Error downloading https://www.discogs.com/artist/1528-Abe-Duque?type=Unofficial
Error downloading https://www.discogs.com/artist/172128-The-Firm?type=Unofficial
Error downloading https://www.discogs.com/artist/295028-Wang-Fu-Ling?type=Unofficial
Error downloading https://www.discogs.com/artist/397328-Millie-Small?type=Unofficial
Error downloading https://www.discogs.com/artist/659328-The-King-Khan-BBQ-Show?type=Unofficial
Error downloading https://www.discogs.com/artist/82828-Android-Lust?type=Unofficial
Error downloading https://www.discogs.com/artist/1489428-Tony-Dangerfield-The-Thrills?type=Unofficial
=========================   discogs   =========================
  ===> Found 7581 previous data for ModVal=29
Found 310 artists to get
Error downloading https://www.discogs.com/artist/20229-Technique?type=Unofficial
Error downloading https://www.discogs.com/artist/218429-Inoyama-Land?type=Unofficial
Error downloading https://www.discogs.com/artist/2227229-Ray-Conniff-His-Singers-His-Orchestra-And-Chorus?type=Unofficial
Error downloading https://www.discogs.com/artist/2577229-Rouge-13?type=Unofficial
Error downloading https://www.discogs.com/artist/258129-Jimmy-Lyons-2?type=Unofficial
Error downloading https://www.discogs.com/artist/319329-Napz?type=Unofficial
Error downloading https://www.discogs.com/artist/33229-Leon?type=Unofficial
Error downloading https://www.discogs.com/artist/39329-The-Beat-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/435529-Sleipnir?type=Unofficial
Error downloading https://www.discogs.com/artist/48929-Jeffee?type=Unofficial
Error downloading https://www.discogs.com/artist/62029-Sandman-2?type=Unofficial
Error downloading https://www.discogs.com/artist/731829-Ocarina-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7620 previous data for ModVal=30
Found 306 artists to get
Error downloading https://www.discogs.com/artist/1248130-Dr-Woggle-The-Radio?type=Unofficial
Error downloading https://www.discogs.com/artist/1468130-Mars-19?type=Unofficial
Error downloading https://www.discogs.com/artist/2561830-Rebel-Drones?type=Unofficial
Error downloading https://www.discogs.com/artist/27330-Visual?type=Unofficial
Error downloading https://www.discogs.com/artist/3030-Manna?type=Unofficial
Error downloading https://www.discogs.com/artist/341030-Glen-Goldsmith?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/30/unofficial/4007830.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/4146830-Ben-Poole-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7533 previous data for ModVal=31
Found 326 artists to get
Error downloading https://www.discogs.com/artist/1120631-La-Bottega-DellArte?type=Unofficial
Error downloading https://www.discogs.com/artist/131-Dance-Fever?type=Unofficial
Error downloading https://www.discogs.com/artist/136131-Music-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/1550831-%C3%90%C2%9D%C3%90%C2%B5-%C3%90%C2%98%C3%90%C2%B3%C3%91%C2%80%C3%91%C2%83%C3%91%C2%88%C3%90%C2%BA%C3%90%C2%B8?type=Unofficial
Error downloading https://www.discogs.com/artist/171531-Blacksmoke?type=Unofficial
Error downloading https://www.discogs.com/artist/2012731-Reuben-Sawyer?type=Unofficial
Error downloading https://www.discogs.com/artist/233631-Big-Jim-Slade?type=Unofficial
Error downloading https://www.discogs.com/artist/298931-Joe-Harriott?type=Unofficial
Error downloading https://www.discogs.com/artist/567231-Belinda-Lipscomb?type=Unofficial
Error downloading https://www.discogs.com/artist/7731-L-Double?type=Unofficial
Error downloading https://www.discogs.com/artist/83931-Jay-Strongman?type=Unofficial
=========================   discogs   =========================
  ===> Found 7624 previous data for ModVal=32
Found 345 artists to get
Error downloading https://www.discogs.com/artist/1003232-Dept-Of-Edu?type=Unofficial
Error downloading https://www.discogs.com/artist/203732-Rob-One?type=Unofficial
Error downloading https://www.discogs.com/artist/27832-Nasa?type=Unofficial
Error downloading https://www.discogs.com/artist/5004432-Exizt?type=Unofficial
Error downloading https://www.discogs.com/artist/510332-Bourvil?type=Unofficial
Error downloading https://www.discogs.com/artist/66732-Alva-Noto-Ryuichi-Sakamoto?type=Unofficial
=========================   discogs   =========================
  ===> Found 7602 previous data for ModVal=33
Found 351 artists to get
Error downloading https://www.discogs.com/artist/1295433-Flax-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1633-Ant-Miles?type=Unofficial
Error downloading https://www.discogs.com/artist/37233-Brian-Tony-Gold?type=Unofficial
Error downloading https://www.discogs.com/artist/41233-Jazzy-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/4376733-Willie-Egans-And-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/447033-Kutmah?type=Unofficial
Error downloading https://www.discogs.com/artist/5674233-Morbid-Mayhem?type=Unofficial
=========================   discogs   =========================
  ===> Found 7592 previous data for ModVal=34
Found 310 artists to get
Error downloading https://www.discogs.com/artist/133634-RMB-and-Friends?type=Unofficial
Error downloading https://www.discogs.com/artist/2031634-Mahsun-K%C4%B1rm%C4%B1z%C4%B1g%C3%BCl?type=Unofficial
Error downloading https://www.discogs.com/artist/353934-Pressure-Release?type=Unofficial
Error downloading https://www.discogs.com/artist/3563234-T%C3%83%C2%A4htiportti-2?type=Unofficial
Error downloading https://www.discogs.com/artist/3727234-Dahna?type=Unofficial
Error downloading https://www.discogs.com/artist/408634-The-Latin-Souls?type=Unofficial
Error downloading https://www.discogs.com/artist/4734-Twisted-Anger?type=Unofficial
Error downloading https://www.discogs.com/artist/5318234-Dj-Gloor?type=Unofficial
Error downloading https://www.discogs.com/artist/559034-Sleep-Clinic?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/34/unofficial/2444134.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7438 previous data for ModVal=35
Found 334 artists to get
Error downloading https://www.discogs.com/artist/108235-Silent-Running?type=Unofficial
Error downloading https://www.discogs.com/artist/1635-Stakka-KTee?type=Unofficial
Error downloading https://www.discogs.com/artist/229035-Final-Edition?type=Unofficial
Error downloading https://www.discogs.com/artist/2652135-Christophe-H%C3%A9ral?type=Unofficial
Error downloading https://www.discogs.com/artist/319935-L%C3%BAcio-Alves?type=Unofficial
Error downloading https://www.discogs.com/artist/378335-Maurizio-Belladonna?type=Unofficial
Error downloading https://www.discogs.com/artist/401635-Carla-Boni?type=Unofficial
Error downloading https://www.discogs.com/artist/4106635-Earforce-2?type=Unofficial
Error downloading https://www.discogs.com/artist/4535-Lucid?type=Unofficial
Error downloading https://www.discogs.com/artist/46235-Global?type=Unofficial
Error downloading https://www.discogs.com/artist/632835-Chiodos?type=Unofficial
Error downloading https://www.discogs.com/artist/7038535-Obscure-Obsolete?type=Unofficial
=========================   discogs   =========================
  ===> Found 7683 previous data for ModVal=36
Found 347 artists to get
Error downloading https://www.discogs.com/artist/102736-Big-Brother?type=Unofficial
Error downloading https://www.discogs.com/artist/12936-Shanie?type=Unofficial
Error downloading https://www.discogs.com/artist/1358536-Stray-8?type=Unofficial
Error downloading https://www.discogs.com/artist/148836-Red-3?type=Unofficial
Error downloading https://www.discogs.com/artist/264336-YUP?type=Unofficial
Error downloading https://www.discogs.com/artist/282436-Bernt-Rosengren?type=Unofficial
Error downloading https://www.discogs.com/artist/29336-DJ-Vitamin-D?type=Unofficial
Error downloading https://www.discogs.com/artist/33936-The-Other-Two?type=Unofficial
Error downloading https://www.discogs.com/artist/4036036-Yunomi?type=Unofficial
Error downloading https://www.discogs.com/artist/575836-Harry-Arnold?type=Unofficial
Error downloading https://www.discogs.com/artist/830636-Gust-Burns?type=Unofficial
Error downloading https://www.discogs.com/artist/92536-Jes-Brieden?type=Unofficial
Error downloading https://www.discogs.com/artist/944636-Mr-Strom?type=Unofficial
Error downloading https://www.discogs.com/artist/99236-Fugitive-2?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/36/unofficial/16336.p (force=False)
  --> This file is 16.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7537 previous data for ModVal=37
Found 332 artists to get
Error downloading https://www.discogs.com/artist/1018437-Moon-Wiring-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/16237-Dakota?type=Unofficial
Error downloading https://www.discogs.com/artist/209437-Lee-Spencer?type=Unofficial
Error downloading https://www.discogs.com/artist/299037-Johnny-Barnes?type=Unofficial
Error downloading https://www.discogs.com/artist/355737-Jonathan-Goldman?type=Unofficial
Error downloading https://www.discogs.com/artist/430437-Mark-Fox?type=Unofficial
Error downloading https://www.discogs.com/artist/48937-Lisahall?type=Unofficial
Error downloading https://www.discogs.com/artist/5443937-Hazel-Mccollum-The-Tren-Teens?type=Unofficial
Error downloading https://www.discogs.com/artist/572337-The-Maskman-And-The-Agents?type=Unofficial
Error downloading https://www.discogs.com/artist/631637-Spectrasoul?type=Unofficial
Error downloading https://www.discogs.com/artist/74937-Grunt?type=Unofficial
=========================   discogs   =========================
  ===> Found 7522 previous data for ModVal=38
Found 310 artists to get
Error downloading https://www.discogs.com/artist/1038138-Social-Disco-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/1198438-Kollegah?type=Unofficial
Error downloading https://www.discogs.com/artist/13738-Numbers?type=Unofficial
Error downloading https://www.discogs.com/artist/3738-SETI?type=Unofficial
Error downloading https://www.discogs.com/artist/41438-M-Factor?type=Unofficial
Error downloading https://www.discogs.com/artist/470538-Matt-Hollywood?type=Unofficial
Error downloading https://www.discogs.com/artist/603338-Roland-Kayn?type=Unofficial
=========================   discogs   =========================
  ===> Found 7537 previous data for ModVal=39
Found 312 artists to get
Error downloading https://www.discogs.com/artist/1144439-The-Ziggens?type=Unofficial
Error downloading https://www.discogs.com/artist/1706439-Chuck-E-Meyers?type=Unofficial
Error downloading https://www.discogs.com/artist/2573939-Andy-Biersack?type=Unofficial
Error downloading https://www.discogs.com/artist/3164639-P%C3%A1l-Esterh%C3%A1zy?type=Unofficial
Error downloading https://www.discogs.com/artist/391039-The-Empires?type=Unofficial
Error downloading https://www.discogs.com/artist/430739-Mike-Waterson?type=Unofficial
Error downloading https://www.discogs.com/artist/5181339-Jimmy-Gilmore-The-Fireballs?type=Unofficial
Error downloading https://www.discogs.com/artist/56639-Skool-Of-Thought?type=Unofficial
Error downloading https://www.discogs.com/artist/839-Deepside?type=Unofficial
=========================   discogs   =========================
  ===> Found 7553 previous data for ModVal=40
Found 343 artists to get
Error downloading https://www.discogs.com/artist/1205840-Future-Islands?type=Unofficial
Error downloading https://www.discogs.com/artist/1453440-Poppis%21?type=Unofficial
Error downloading https://www.discogs.com/artist/1940840-hotliners?type=Unofficial
Error downloading https://www.discogs.com/artist/1961240-Laurent-Naouri?type=Unofficial
Error downloading https://www.discogs.com/artist/280240-Marlo-Hoogstraten?type=Unofficial
Error downloading https://www.discogs.com/artist/37940-Chris-Connelly?type=Unofficial
Error downloading https://www.discogs.com/artist/639840-Franz-Lehrndorfer?type=Unofficial
Error downloading https://www.discogs.com/artist/766540-Ruffneck-Featuring-Yavahn?type=Unofficial
Error downloading https://www.discogs.com/artist/82540-Super-3?type=Unofficial
Error downloading https://www.discogs.com/artist/838040-Kammerorchester-Berlin?type=Unofficial
Error downloading https://www.discogs.com/artist/8540-Jerry-Ropero?type=Unofficial
=========================   discogs   =========================
  ===> Found 7759 previous data for ModVal=41
Found 349 artists to get
Error downloading https://www.discogs.com/artist/1376041-Dj-Evo?type=Unofficial
Error downloading https://www.discogs.com/artist/1641-Fortran?type=Unofficial
Error downloading https://www.discogs.com/artist/184441-Bernard-Heidsieck?type=Unofficial
Error downloading https://www.discogs.com/artist/2087041-Judas-9?type=Unofficial
Error downloading https://www.discogs.com/artist/2147641-DJ-Guilty?type=Unofficial
Error downloading https://www.discogs.com/artist/2463841-Sacred-Heart?type=Unofficial
Error downloading https://www.discogs.com/artist/2469341-Creation-Of-Sunlight?type=Unofficial
Error downloading https://www.discogs.com/artist/2644641-Teddy-McRae-Orchestra?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/41/unofficial/351541.p (force=False)
  --> This file is 26.5kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/357441-Eyedea-Abilities?type=Unofficial
Error downloading https://www.discogs.com/artist/611741-Freddie-Gorman?type=Unofficial
Error downloading https://www.discogs.com/artist/751941-Erno-Rapee?type=Unofficial
=========================   discogs   =========================
  ===> Found 7607 previous data for ModVal=42
Found 314 artists to get
Error downloading https://www.discogs.com/artist/1198742-Dennis-Day?type=Unofficial
Error downloading https://www.discogs.com/artist/1395642-Frivolvol?type=Unofficial
Error downloading https://www.discogs.com/artist/1835542-Dj-Diggz?type=Unofficial
Error downloading https://www.discogs.com/artist/257142-The-Flesh-Eaters?type=Unofficial
Error downloading https://www.discogs.com/artist/292442-Justin-Hinds?type=Unofficial
Error downloading https://www.discogs.com/artist/36142-Vernessa-Mitchell?type=Unofficial
Error downloading https://www.discogs.com/artist/429942-Bone-Symphony?type=Unofficial
Error downloading https://www.discogs.com/artist/529142-Bosse-Skoglund?type=Unofficial
Error downloading https://www.discogs.com/artist/5423542-Twin-Sisters-Productions-Inc?type=Unofficial
Error downloading https://www.discogs.com/artist/5442-Casino-Versus-Japan?type=Unofficial
Error downloading https://www.discogs.com/artist/703742-Rod-Morris-2?type=Unofficial
Error downloading https://www.discogs.com/artist/790742-Tom-Reeves?type=Unofficial
Error downloading https://www.discogs.com/artist/971242-Calcutta?type=Unofficial
=========================   discogs   =========================
  ===> Found 7622 previous data for ModVal=43
Found 339 artists to get
Error downloading https://www.discogs.com/artist/1046943-Hamburger-Kammerorchester?type=Unofficial
Error downloading https://www.discogs.com/artist/3085043-DJ-Shame-2?type=Unofficial
Error downloading https://www.discogs.com/artist/499643-Fausto-Rossi?type=Unofficial
Error downloading https://www.discogs.com/artist/648343-Al-Smith?type=Unofficial
Error downloading https://www.discogs.com/artist/6956543-Hideamans-Way?type=Unofficial
Error downloading https://www.discogs.com/artist/83143-Pebbles?type=Unofficial
=========================   discogs   =========================
  ===> Found 7453 previous data for ModVal=44
Found 345 artists to get
Error downloading https://www.discogs.com/artist/1104044-Have-A-Nice-Life?type=Unofficial
Error downloading https://www.discogs.com/artist/270544-Danny-Janssen?type=Unofficial
Error downloading https://www.discogs.com/artist/45644-John-Wesley-Harding?type=Unofficial
Error downloading https://www.discogs.com/artist/748944-Jump-With-Joey?type=Unofficial
=========================   discogs   =========================
  ===> Found 7416 previous data for ModVal=45
Found 337 artists to get
Error downloading https://www.discogs.com/artist/111545-DJ-Jeroenski?type=Unofficial
Error downloading https://www.discogs.com/artist/1439545-Kamal-Ahmed?type=Unofficial
Error downloading https://www.discogs.com/artist/190445-Jean-Louis-Aubert?type=Unofficial
Error downloading https://www.discogs.com/artist/2023545-Orchester-Carl-Woitschach?type=Unofficial
Error downloading https://www.discogs.com/artist/252045-Jill-Sobule?type=Unofficial
Error downloading https://www.discogs.com/artist/2891145-Ewan-Dobson?type=Unofficial
Error downloading https://www.discogs.com/artist/293445-Cynthia-Harrell?type=Unofficial
Error downloading https://www.discogs.com/artist/34945-Madoka?type=Unofficial
=========================   discogs   =========================
  ===> Found 7493 previous data for ModVal=46
Found 328 artists to get
Error downloading https://www.discogs.com/artist/10046-DJ-Pulse?type=Unofficial
Error downloading https://www.discogs.com/artist/104746-Count-Bass-D?type=Unofficial
Error downloading https://www.discogs.com/artist/1102746-Los-Monaguillosh?type=Unofficial
Error downloading https://www.discogs.com/artist/122446-Daniel-Ash?type=Unofficial
Error downloading https://www.discogs.com/artist/1788046-%CE%97%CE%BB%CE%AF%CE%B1%CF%82-%CE%91%CF%83%CE%B2%CE%B5%CF%83%CF%84%CF%8C%CF%80%CE%BF%CF%85%CE%BB%CE%BF%CF%82?type=Unofficial
Error downloading https://www.discogs.com/artist/219746-Street-Beat?type=Unofficial
Error downloading https://www.discogs.com/artist/263346-Nikki-Sudden?type=Unofficial
Error downloading https://www.discogs.com/artist/268846-The-Stones?type=Unofficial
Error downloading https://www.discogs.com/artist/295346-Trevor-Lucas?type=Unofficial
Error downloading https://www.discogs.com/artist/332446-Basto%21?type=Unofficial
Error downloading https://www.discogs.com/artist/381846-Mannish?type=Unofficial
Error downloading https://www.discogs.com/artist/59946-Loscil?type=Unofficial
Error downloading https://www.discogs.com/artist/64546-Funny-F?type=Unofficial
Error downloading https://www.discogs.com/artist/7810846-Notorious-BIG-50-Cent?type=Unofficial
Error downloading https://www.discogs.com/artist/8046-Marlow?type=Unofficial
Error downloading https://www.discogs.com/artist/94146-Too-Strong?type=Unofficial
=========================   discogs   =========================
  ===> Found 7413 previous data for ModVal=47
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1508547-2000-Mix?type=Unofficial
Error downloading https://www.discogs.com/artist/161447-DJ-Ryow?type=Unofficial
Error downloading https://www.discogs.com/artist/2193747-Deep-Disco?type=Unofficial
Error downloading https://www.discogs.com/artist/2405647-Ultrademon?type=Unofficial
Error downloading https://www.discogs.com/artist/278347-Frank-N-Dank?type=Unofficial
Error downloading https://www.discogs.com/artist/341947-David-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/371747-Rejuvenate?type=Unofficial
Error downloading https://www.discogs.com/artist/701547-Cozi?type=Unofficial
Error downloading https://www.discogs.com/artist/734847-Garrett-Klahn?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/47/unofficial/1685347.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7706 previous data for ModVal=48
Found 308 artists to get
Error downloading https://www.discogs.com/artist/16248-Stranger?type=Unofficial
Error downloading https://www.discogs.com/artist/2565548-Girls-With-Guitars?type=Unofficial
Error downloading https://www.discogs.com/artist/2722848-Various-Godfather-EP?type=Unofficial
Error downloading https://www.discogs.com/artist/2850248-The-Strand-6?type=Unofficial
Error downloading https://www.discogs.com/artist/3448-Lab-Rats?type=Unofficial
Error downloading https://www.discogs.com/artist/4668248-Kis-Grof%C3%83%C2%B3?type=Unofficial
Error downloading https://www.discogs.com/artist/696048-The-Superlatives-3?type=Unofficial
=========================   discogs   =========================
  ===> Found 7655 previous data for ModVal=49
Found 313 artists to get
Error downloading https://www.discogs.com/artist/110949-Punish-Yourself?type=Unofficial
Error downloading https://www.discogs.com/artist/1247149-The-Pride-3?type=Unofficial
Error downloading https://www.discogs.com/artist/131049-Paddy-Kingsland?type=Unofficial
Error downloading https://www.discogs.com/artist/272449-Cure?type=Unofficial
Error downloading https://www.discogs.com/artist/761549-Svart-Framtid?type=Unofficial
Error downloading https://www.discogs.com/artist/9949-DJ-Technics?type=Unofficial
=========================   discogs   =========================
  ===> Found 7604 previous data for ModVal=50
Found 332 artists to get
Error downloading https://www.discogs.com/artist/201450-Solitude-FX?type=Unofficial
Error downloading https://www.discogs.com/artist/2559550-Yuu?type=Unofficial
Error downloading https://www.discogs.com/artist/27650-Reefer-Decree?type=Unofficial
Error downloading https://www.discogs.com/artist/332450-Jimmy-Lewis-2?type=Unofficial
Error downloading https://www.discogs.com/artist/371750-108-3?type=Unofficial
Error downloading https://www.discogs.com/artist/774750-Inoki?type=Unofficial
=========================   discogs   =========================
  ===> Found 7678 previous data for ModVal=51
Found 324 artists to get
Error downloading https://www.discogs.com/artist/181351-GLC?type=Unofficial
Error downloading https://www.discogs.com/artist/2045151-Kahn-5?type=Unofficial
Error downloading https://www.discogs.com/artist/2055951-John-Mellencamp?type=Unofficial
Error downloading https://www.discogs.com/artist/243951-Bobby-Franklins-Insanity?type=Unofficial
Error downloading https://www.discogs.com/artist/325951-Mask-5?type=Unofficial
Error downloading https://www.discogs.com/artist/505451-Nino-Oliviero?type=Unofficial
Error downloading https://www.discogs.com/artist/5348051-Bruce-Springsteen-Bon-Jovi?type=Unofficial
Error downloading https://www.discogs.com/artist/651-Digital?type=Unofficial
Error downloading https://www.discogs.com/artist/7451-Cusp?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/51/unofficial/1234651.p (force=False)
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/51/unofficial/5223351.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7682 previous data for ModVal=52
Found 372 artists to get
Error downloading https://www.discogs.com/artist/1054252-Franco-Califano?type=Unofficial
Error downloading https://www.discogs.com/artist/124052-Maxim-Sinclair?type=Unofficial
Error downloading https://www.discogs.com/artist/430752-Lal-Waterson?type=Unofficial
Error downloading https://www.discogs.com/artist/6346252-Herbie-Hancock-headhunters?type=Unofficial
Error downloading https://www.discogs.com/artist/665552-Various-3?type=Unofficial
Error downloading https://www.discogs.com/artist/962052-Skirt?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/52/unofficial/667752.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7651 previous data for ModVal=53
Found 331 artists to get
Error downloading https://www.discogs.com/artist/124053-Paul-Benjamin?type=Unofficial
Error downloading https://www.discogs.com/artist/2853-Kemal-Rob-Data?type=Unofficial
Error downloading https://www.discogs.com/artist/292153-The-Extras?type=Unofficial
Error downloading https://www.discogs.com/artist/349953-Prinz-Porno?type=Unofficial
Error downloading https://www.discogs.com/artist/3622953-Richard-Moreland?type=Unofficial
Error downloading https://www.discogs.com/artist/6360753-Ruth-Brown-And-Her-Rhythmakers-With-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/707253-Walter-Rhodes?type=Unofficial
Error downloading https://www.discogs.com/artist/856653-Schubert-3?type=Unofficial
=========================   discogs   =========================
  ===> Found 7727 previous data for ModVal=54
Found 319 artists to get
Error downloading https://www.discogs.com/artist/1107254-Gnod?type=Unofficial
Error downloading https://www.discogs.com/artist/150554-Guilty-Simpson?type=Unofficial
Error downloading https://www.discogs.com/artist/1621054-DJ-NikOne?type=Unofficial
Error downloading https://www.discogs.com/artist/2109254-Mission-Infect?type=Unofficial
Error downloading https://www.discogs.com/artist/2854-Stakka-Skynet?type=Unofficial
Error downloading https://www.discogs.com/artist/3371254-July-Talk?type=Unofficial
Error downloading https://www.discogs.com/artist/4332854-JJBurner?type=Unofficial
Error downloading https://www.discogs.com/artist/446154-Albert-Kuvezin?type=Unofficial
Error downloading https://www.discogs.com/artist/4954-Ryme-Tyme?type=Unofficial
=========================   discogs   =========================
  ===> Found 7677 previous data for ModVal=55
Found 333 artists to get
Error downloading https://www.discogs.com/artist/1028055-Tramteatret?type=Unofficial
Error downloading https://www.discogs.com/artist/122055-Point-Blank-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2211255-TEST-9?type=Unofficial
Error downloading https://www.discogs.com/artist/2779455-Soulspell?type=Unofficial
Error downloading https://www.discogs.com/artist/2790255-BEP?type=Unofficial
Error downloading https://www.discogs.com/artist/296255-Jimmy-Destri?type=Unofficial
Error downloading https://www.discogs.com/artist/3555-Negrocan?type=Unofficial
Error downloading https://www.discogs.com/artist/404655-Mistah-FAB?type=Unofficial
Error downloading https://www.discogs.com/artist/430555-The-Sperm?type=Unofficial
Error downloading https://www.discogs.com/artist/446155-Yat-Kha?type=Unofficial
Error downloading https://www.discogs.com/artist/69955-Dover?type=Unofficial
Error downloading https://www.discogs.com/artist/90855-Heliotrope?type=Unofficial
=========================   discogs   =========================
  ===> Found 7620 previous data for ModVal=56
Found 354 artists to get
Error downloading https://www.discogs.com/artist/1220456-Riff-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2223556-Academics-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2642356-Dr-Livingstone-2?type=Unofficial
Error downloading https://www.discogs.com/artist/306056-Normil-Hawaiians?type=Unofficial
Error downloading https://www.discogs.com/artist/374856-Quarterflash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/56/unofficial/3900856.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/560556-Elijah-Williams-Jr?type=Unofficial
Error downloading https://www.discogs.com/artist/818356-Primary-1?type=Unofficial
Error downloading https://www.discogs.com/artist/858756-The-Royal-Stockholm-Philharmonic-Orchestra?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/56/unofficial/264456.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7600 previous data for ModVal=57
Found 347 artists to get
Error downloading https://www.discogs.com/artist/1634557-Lawson-Haggart-Jazz-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/266857-Chokehold?type=Unofficial
Error downloading https://www.discogs.com/artist/267857-Patty-Smyth?type=Unofficial
Error downloading https://www.discogs.com/artist/434457-Preacher-3?type=Unofficial
Error downloading https://www.discogs.com/artist/4493057-Ian-Gillan-And-The-Javelins?type=Unofficial
Error downloading https://www.discogs.com/artist/4982757-Theo-Shanab?type=Unofficial
Error downloading https://www.discogs.com/artist/6312757-Ruth-Brown-With-Orchestra?type=Unofficial
=========================   discogs   =========================
  ===> Found 7542 previous data for ModVal=58
Found 318 artists to get
Error downloading https://www.discogs.com/artist/12758-London-Fiesta?type=Unofficial
Error downloading https://www.discogs.com/artist/1400458-The-Honeydreamers?type=Unofficial
Error downloading https://www.discogs.com/artist/148058-Exile-6?type=Unofficial
Error downloading https://www.discogs.com/artist/2312258-DJ-Quest-6?type=Unofficial
Error downloading https://www.discogs.com/artist/2356058-The-First-National-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/249758-Ziggy-Marley?type=Unofficial
Error downloading https://www.discogs.com/artist/27758-4-Clubbers?type=Unofficial
Error downloading https://www.discogs.com/artist/29358-Suzanne-Palmer?type=Unofficial
Error downloading https://www.discogs.com/artist/3343158-Jorge-Cardoso-2?type=Unofficial
Error downloading https://www.discogs.com/artist/655958-DJ-Basilisk?type=Unofficial
=========================   discogs   =========================
  ===> Found 7732 previous data for ModVal=59
Found 304 artists to get
Error downloading https://www.discogs.com/artist/133959-Tic-Tac-Toe?type=Unofficial
Error downloading https://www.discogs.com/artist/13559-Elements?type=Unofficial
Error downloading https://www.discogs.com/artist/1742559-IA-Batiste?type=Unofficial
Error downloading https://www.discogs.com/artist/394459-Ismael-Quintana?type=Unofficial
Error downloading https://www.discogs.com/artist/4200559-American-Pulverizer?type=Unofficial
Error downloading https://www.discogs.com/artist/700759-The-Spits?type=Unofficial
Error downloading https://www.discogs.com/artist/799359-Miles-Kane?type=Unofficial
Error downloading https://www.discogs.com/artist/843259-Luis-Cilia?type=Unofficial
Error downloading https://www.discogs.com/artist/996059-Pete-Bonus?type=Unofficial
=========================   discogs   =========================
  ===> Found 7656 previous data for ModVal=60
Found 335 artists to get
Error downloading https://www.discogs.com/artist/1166760-Alonzo-Scales?type=Unofficial
Error downloading https://www.discogs.com/artist/2333160-Chorus-And-Orchestra-Of-The-Teatro-Di-San-Carlo-Di-Napoli?type=Unofficial
Error downloading https://www.discogs.com/artist/240660-Teej?type=Unofficial
Error downloading https://www.discogs.com/artist/348860-IQ-3?type=Unofficial
Error downloading https://www.discogs.com/artist/5521260-Doukutsu-Monogatari?type=Unofficial
Error downloading https://www.discogs.com/artist/669060-Bride?type=Unofficial
=========================   discogs   =========================
  ===> Found 7595 previous data for ModVal=61
Found 325 artists to get
Error downloading https://www.discogs.com/artist/1851061-Lil-Ceasar-2?type=Unofficial
Error downloading https://www.discogs.com/artist/287861-Tony-Garnier?type=Unofficial
Error downloading https://www.discogs.com/artist/377361-The-Finn-Brothers?type=Unofficial
Error downloading https://www.discogs.com/artist/429961-The-Jacobites?type=Unofficial
Error downloading https://www.discogs.com/artist/4488061-The-Bittersweets-5?type=Unofficial
=========================   discogs   =========================
  ===> Found 7711 previous data for ModVal=62
Found 345 artists to get
Error downloading https://www.discogs.com/artist/11762-The-Dronez?type=Unofficial
Error downloading https://www.discogs.com/artist/1551762-The-Live-Experience-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/165262-Pont-Aeri?type=Unofficial
Error downloading https://www.discogs.com/artist/200962-Animals?type=Unofficial
Error downloading https://www.discogs.com/artist/20562-Reggie-Hall?type=Unofficial
Error downloading https://www.discogs.com/artist/23762-Gabi-Delgado?type=Unofficial
Error downloading https://www.discogs.com/artist/334662-Caustic?type=Unofficial
Error downloading https://www.discogs.com/artist/369762-Doomsday-Productions?type=Unofficial
Error downloading https://www.discogs.com/artist/3830862-Wenge-Musica-BCBG?type=Unofficial
Error downloading https://www.discogs.com/artist/488362-Giacomo-Meyerbeer?type=Unofficial
Error downloading https://www.discogs.com/artist/665162-Scythe?type=Unofficial
Error downloading https://www.discogs.com/artist/6662-DJ-Reality?type=Unofficial
Error downloading https://www.discogs.com/artist/863562-Bread-And-Water-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7701 previous data for ModVal=63
Found 355 artists to get
Error downloading https://www.discogs.com/artist/1477463-Bedlam-7?type=Unofficial
Error downloading https://www.discogs.com/artist/1809563-The-Don-Harrison-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/233863-Private-Lives?type=Unofficial
Error downloading https://www.discogs.com/artist/2595863-Lucky-Carmichael?type=Unofficial
Error downloading https://www.discogs.com/artist/267063-Heaven-Shall-Burn?type=Unofficial
Error downloading https://www.discogs.com/artist/28063-Fred-Jorio?type=Unofficial
Error downloading https://www.discogs.com/artist/3478363-Tuva-2?type=Unofficial
Error downloading https://www.discogs.com/artist/355463-Recover?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/63/unofficial/3568763.p (force=False)
  --> This file is 24.2kB.
Done. Sleeping for 2 seconds
Error downloading https://www.discogs.com/artist/499363-Alexander-Spence?type=Unofficial
Error downloading https://www.discogs.com/artist/531563-DJ-%D0%9A%D0%B8%D1%80%D0%B8%D0%BB%D0%BEff?type=Unofficial
Error downloading https://www.discogs.com/artist/61563-Hard-Trance?type=Unofficial
Error downloading https://www.discogs.com/artist/727663-Shigeru-Matsuzaki?type=Unofficial
Error downloading https://www.discogs.com/artist/843463-The-Waikikis?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/63/unofficial/1499663.p (force=False)
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7757 previous data for ModVal=64
Found 338 artists to get
Error downloading https://www.discogs.com/artist/1088064-Dana-Cooper?type=Unofficial
Error downloading https://www.discogs.com/artist/1448764-%CE%A7%CE%B1%CE%BF%CF%84%CE%B9%CE%BA%CF%8C-%CE%A4%CE%AD%CE%BB%CE%BF%CF%82?type=Unofficial
Error downloading https://www.discogs.com/artist/339764-Potato-5?type=Unofficial
Error downloading https://www.discogs.com/artist/3450364-Young-Pone?type=Unofficial
Error downloading https://www.discogs.com/artist/512764-Angela-Strehli?type=Unofficial
Error downloading https://www.discogs.com/artist/5229464-Guys-Dolls-5?type=Unofficial
Error downloading https://www.discogs.com/artist/7464-DJs-Unite?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/64/unofficial/2991664.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/64/unofficial/3453464.p (force=False)
  --> This file is 16.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7680 previous data for ModVal=65
Found 296 artists to get
Error downloading https://www.discogs.com/artist/10465-Mask?type=Unofficial
Error downloading https://www.discogs.com/artist/1087265-Ralph-Marterie?type=Unofficial
Error downloading https://www.discogs.com/artist/247365-Ron-Richardson?type=Unofficial
Error downloading https://www.discogs.com/artist/254865-The-Rhythm-Rockers?type=Unofficial
Error downloading https://www.discogs.com/artist/264865-Alana-Dante?type=Unofficial
Error downloading https://www.discogs.com/artist/282265-Autonomous-Zone?type=Unofficial
Error downloading https://www.discogs.com/artist/2835565-Jimmy-Ricks-The-Ravens?type=Unofficial
Error downloading https://www.discogs.com/artist/4013265-Johnny-Mastro-And-The-Mamas-Boys?type=Unofficial
Error downloading https://www.discogs.com/artist/4265-Dean-Coleman?type=Unofficial
Error downloading https://www.discogs.com/artist/4266465-Jake-Chudnow?type=Unofficial
Error downloading https://www.discogs.com/artist/54965-NDX?type=Unofficial
=========================   discogs   =========================
  ===> Found 7632 previous data for ModVal=66
Found 325 artists to get
Error downloading https://www.discogs.com/artist/10466-Swabe?type=Unofficial
Error downloading https://www.discogs.com/artist/1106266-Agust%C3%ADn-Barrios-Mangor%C3%A9?type=Unofficial
Error downloading https://www.discogs.com/artist/147666-Yana-Kay?type=Unofficial
Error downloading https://www.discogs.com/artist/4318566-Pearl-Handle-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/5070666-Castiel-2?type=Unofficial
=========================   discogs   =========================
  ===> Found 7728 previous data for ModVal=67
Found 359 artists to get
Error downloading https://www.discogs.com/artist/15467-CoH?type=Unofficial
Error downloading https://www.discogs.com/artist/20367-Legacy?type=Unofficial
Error downloading https://www.discogs.com/artist/212667-NgaFsh?type=Unofficial
Error downloading https://www.discogs.com/artist/216667-Mean-Green?type=Unofficial
Error downloading https://www.discogs.com/artist/263767-Beyond-Description?type=Unofficial
Error downloading https://www.discogs.com/artist/34067-Pete-Hammond?type=Unofficial
Error downloading https://www.discogs.com/artist/630867-William-Bell-2?type=Unofficial
Error downloading https://www.discogs.com/artist/648667-Sylvia-7?type=Unofficial
Error downloading https://www.discogs.com/artist/654667-The-LoveCrave?type=Unofficial
Error downloading https://www.discogs.com/artist/70467-Xorcist?type=Unofficial
Error downloading https://www.discogs.com/artist/887967-Ray-Scott-2?type=Unofficial
Error downloading https://www.discogs.com/artist/950167-Rich-Stearns?type=Unofficial
Error downloading https://www.discogs.com/artist/99567-Toms-Project?type=Unofficial
=========================   discogs   =========================
  ===> Found 7474 previous data for ModVal=68
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1056568-Zagreba%C4%8Dki-Solisti?type=Unofficial
Error downloading https://www.discogs.com/artist/1059168-Mari-Nakamoto?type=Unofficial
Error downloading https://www.discogs.com/artist/134068-Distant-Drums?type=Unofficial
Error downloading https://www.discogs.com/artist/2505468-Giorgio-Canali-Rosso-Fuoco?type=Unofficial
Error downloading https://www.discogs.com/artist/3161968-Ivory-Tiger?type=Unofficial
Error downloading https://www.discogs.com/artist/689168-Robert-Byrd?type=Unofficial
Error downloading https://www.discogs.com/artist/78668-The-Deele?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/68/unofficial/53868.p (force=False)
  --> This file is 17.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7618 previous data for ModVal=69
Found 329 artists to get
Error downloading https://www.discogs.com/artist/2195669-Rocky-Campbell?type=Unofficial
Error downloading https://www.discogs.com/artist/2681469-Icewind?type=Unofficial
Error downloading https://www.discogs.com/artist/278269-XMen?type=Unofficial
Error downloading https://www.discogs.com/artist/280269-Sean-Nelson?type=Unofficial
Error downloading https://www.discogs.com/artist/430769-Cheo-Feliciano?type=Unofficial
Error downloading https://www.discogs.com/artist/818569-Les-Baxter-His-Chorus-And-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/850869-Fly-Bi-Night?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/69/unofficial/360369.p (force=False)
  --> This file is 17.2kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7536 previous data for ModVal=70
Found 366 artists to get
Error downloading https://www.discogs.com/artist/13870-The-B-Boys?type=Unofficial
Error downloading https://www.discogs.com/artist/2442770-%CE%A3%CE%B1%CE%BC%CF%80%CF%81%CE%AF%CE%BD%CE%B1?type=Unofficial
Error downloading https://www.discogs.com/artist/271770-The-Macc-Lads?type=Unofficial
Error downloading https://www.discogs.com/artist/3773570-Diamond-Ortiz?type=Unofficial
Error downloading https://www.discogs.com/artist/5775970-Elton-John-Billy-Joel?type=Unofficial
Error downloading https://www.discogs.com/artist/599270-Roy-Montrell?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/70/unofficial/4249570.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7573 previous data for ModVal=71
Found 329 artists to get
Error downloading https://www.discogs.com/artist/1017371-Ebenezer-Obey?type=Unofficial
Error downloading https://www.discogs.com/artist/1271-Tranquility-Bass?type=Unofficial
Error downloading https://www.discogs.com/artist/1318471-Yosuke-Onuma?type=Unofficial
Error downloading https://www.discogs.com/artist/1718771-Yellow-Brick-Road?type=Unofficial
Error downloading https://www.discogs.com/artist/191271-Biddu?type=Unofficial
Error downloading https://www.discogs.com/artist/2734971-Las-Antorchas?type=Unofficial
Error downloading https://www.discogs.com/artist/5588471-Dr-Mario-5?type=Unofficial
Error downloading https://www.discogs.com/artist/6171-Facs?type=Unofficial
Error downloading https://www.discogs.com/artist/770571-Dr-Demento?type=Unofficial
Error downloading https://www.discogs.com/artist/904171-Yumi-Matsuzawa?type=Unofficial
=========================   discogs   =========================
  ===> Found 7585 previous data for ModVal=72
Found 322 artists to get
Error downloading https://www.discogs.com/artist/314972-The-Lincoln-Center-Jazz-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/4235572-Nozomu-Wakais-Destinia?type=Unofficial
Error downloading https://www.discogs.com/artist/5335372-Eri-Lime?type=Unofficial
=========================   discogs   =========================
  ===> Found 7598 previous data for ModVal=73
Found 353 artists to get
Error downloading https://www.discogs.com/artist/1291373-Loka-Nunda?type=Unofficial
Error downloading https://www.discogs.com/artist/1432073-Deceptor-2?type=Unofficial
Error downloading https://www.discogs.com/artist/1600173-Kreton?type=Unofficial
Error downloading https://www.discogs.com/artist/181973-Avalon-4?type=Unofficial
Error downloading https://www.discogs.com/artist/205073-Federation?type=Unofficial
Error downloading https://www.discogs.com/artist/217273-Juhani?type=Unofficial
Error downloading https://www.discogs.com/artist/2403173-Falling-In-Reverse?type=Unofficial
Error downloading https://www.discogs.com/artist/242973-Seiei-Jack?type=Unofficial
Error downloading https://www.discogs.com/artist/416073-New-York-Scratch-Masters?type=Unofficial
Error downloading https://www.discogs.com/artist/450273-?type=Unofficial
Error downloading https://www.discogs.com/artist/84073-Pink-Industry?type=Unofficial
=========================   discogs   =========================
  ===> Found 7651 previous data for ModVal=74
Found 343 artists to get
Error downloading https://www.discogs.com/artist/1111674-John-Berberian?type=Unofficial
Error downloading https://www.discogs.com/artist/1813674-Chuck-U?type=Unofficial
Error downloading https://www.discogs.com/artist/2046974-Hydropneumothorax?type=Unofficial
Error downloading https://www.discogs.com/artist/2645574-Coro-E-Orchestra-DellAccademia-Di-Santa-Cecilia?type=Unofficial
Error downloading https://www.discogs.com/artist/2748774-Yarmulke?type=Unofficial
Error downloading https://www.discogs.com/artist/2948474-TT-34?type=Unofficial
Error downloading https://www.discogs.com/artist/338774-Mike-Joyce?type=Unofficial
Error downloading https://www.discogs.com/artist/341574-Homesick-James?type=Unofficial
Error downloading https://www.discogs.com/artist/4107374-Julian-Wareing?type=Unofficial
Error downloading https://www.discogs.com/artist/529074-The-Sinceros?type=Unofficial
Error downloading https://www.discogs.com/artist/58874-Apollo-XI?type=Unofficial
Error downloading https://www.discogs.com/artist/7374-Wishdokta?type=Unofficial
=========================   discogs   =========================
  ===> Found 7631 previous data for ModVal=75
Found 336 artists to get
Error downloading https://www.discogs.com/artist/1333275-Metz?type=Unofficial
Error downloading https://www.discogs.com/artist/2075-Cloud-9?type=Unofficial
Error downloading https://www.discogs.com/artist/252475-The-Sparkles?type=Unofficial
Error downloading https://www.discogs.com/artist/258175-Mr-Ed-Jumps-The-Gun?type=Unofficial
Error downloading https://www.discogs.com/artist/290775-Marsheaux?type=Unofficial
Error downloading https://www.discogs.com/artist/359475-Henry-Lee-Summer?type=Unofficial
Error downloading https://www.discogs.com/artist/5336975-Capolinea-Internazional-Funk-Banda?type=Unofficial
Error downloading https://www.discogs.com/artist/5420475-The-Sleigh-Ride-Singers?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/75/unofficial/5691275.p (force=False)
  --> This file is 17.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7589 previous data for ModVal=76
Found 344 artists to get
Error downloading https://www.discogs.com/artist/10576-DJ-Noize?type=Unofficial
Error downloading https://www.discogs.com/artist/1085676-Magic-Muscle?type=Unofficial
Error downloading https://www.discogs.com/artist/1756776-Napol%C3%A9on-Coste?type=Unofficial
Error downloading https://www.discogs.com/artist/255376-Bobby-Whitlock?type=Unofficial
Error downloading https://www.discogs.com/artist/3326876-The-Joystix?type=Unofficial
Error downloading https://www.discogs.com/artist/3947676-Shawn-Sahm-And-Prezence?type=Unofficial
Error downloading https://www.discogs.com/artist/510576-Edgar-Winters-White-Trash?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/76/unofficial/4264976.p (force=False)
  --> This file is 17.0kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/76/unofficial/1033576.p (force=False)
  --> This file is 16.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7600 previous data for ModVal=77
Found 341 artists to get
Error downloading https://www.discogs.com/artist/1245777-Britten-Sinfonia?type=Unofficial
Error downloading https://www.discogs.com/artist/14577-Thomas-Penton?type=Unofficial
Error downloading https://www.discogs.com/artist/152277-BabyFace-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/1558977-Mushrooms-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/1606377-Rock-Candy-5?type=Unofficial
Error downloading https://www.discogs.com/artist/2063277-C-Arjun?type=Unofficial
Error downloading https://www.discogs.com/artist/2129077-Infernal-Curse?type=Unofficial
Error downloading https://www.discogs.com/artist/2140877-Liberators?type=Unofficial
Error downloading https://www.discogs.com/artist/262277-Ann-Rabson?type=Unofficial
Error downloading https://www.discogs.com/artist/314877-Morning-Musume?type=Unofficial
Error downloading https://www.discogs.com/artist/345277-DJ-Renat?type=Unofficial
Error downloading https://www.discogs.com/artist/5202377-Big-Duke-4?type=Unofficial
Error downloading https://www.discogs.com/artist/828377-Koroded?type=Unofficial
=========================   discogs   =========================
  ===> Found 7749 previous data for ModVal=78
Found 339 artists to get
Error downloading https://www.discogs.com/artist/112478-Demolition?type=Unofficial
Error downloading https://www.discogs.com/artist/131878-Gene-Dunlap?type=Unofficial
Error downloading https://www.discogs.com/artist/153078-Scottie?type=Unofficial
Error downloading https://www.discogs.com/artist/1752278-Raul-Porchetto?type=Unofficial
Error downloading https://www.discogs.com/artist/2409178-B-Jam-2?type=Unofficial
Error downloading https://www.discogs.com/artist/2601278-Hristo-2?type=Unofficial
Error downloading https://www.discogs.com/artist/389478-The-Highlighters-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/39878-Alice-D-In-Wonderland?type=Unofficial
Error downloading https://www.discogs.com/artist/5081978-Eric-Bibb-And-North-Country-Far-With-Danny-Thompson?type=Unofficial
=========================   discogs   =========================
  ===> Found 7629 previous data for ModVal=79
Found 324 artists to get
Error downloading https://www.discogs.com/artist/1036479-Race-War?type=Unofficial
Error downloading https://www.discogs.com/artist/168579-Nico-4?type=Unofficial
Error downloading https://www.discogs.com/artist/3768579-Djemeia?type=Unofficial
Error downloading https://www.discogs.com/artist/4545579-Fox-McGee?type=Unofficial
Error downloading https://www.discogs.com/artist/876179-LAngelo-Misterioso?type=Unofficial
Error downloading https://www.discogs.com/artist/5945379-Chris-Nisley?type=Unofficial
=========================   discogs   =========================
  ===> Found 7654 previous data for ModVal=80
Found 318 artists to get
Error downloading https://www.discogs.com/artist/1648180-Bad-B-%D0%90%D0%BB%D1%8C%D1%8F%D0%BD%D1%81?type=Unofficial
Error downloading https://www.discogs.com/artist/171780-Jamaaladeen-Tacuma?type=Unofficial
Error downloading https://www.discogs.com/artist/280-Boymerang?type=Unofficial
Error downloading https://www.discogs.com/artist/307580-Norio-Maeda?type=Unofficial
Error downloading https://www.discogs.com/artist/374780-Marc-Mac?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/80/unofficial/4464180.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7789 previous data for ModVal=81
Found 353 artists to get
Error downloading https://www.discogs.com/artist/1213281-Merle-Haggard-And-The-Strangers?type=Unofficial
Error downloading https://www.discogs.com/artist/1425381-New-Tango-Orquestra?type=Unofficial
Error downloading https://www.discogs.com/artist/1446581-2002-GR?type=Unofficial
Error downloading https://www.discogs.com/artist/1555581-Kriek-Balise?type=Unofficial
Error downloading https://www.discogs.com/artist/2062081-Bob-Dylan-With-Tom-Petty-And-The-Heartbreakers?type=Unofficial
Error downloading https://www.discogs.com/artist/227481-DNS-3?type=Unofficial
Error downloading https://www.discogs.com/artist/290581-Big-Joe-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/334081-Hugo-Rignold?type=Unofficial
Error downloading https://www.discogs.com/artist/3466281-Executive-FourMel-Britt?type=Unofficial
Error downloading https://www.discogs.com/artist/4023281-Roy-Ross-And-His-Orchestra?type=Unofficial
Error downloading https://www.discogs.com/artist/43281-Lio?type=Unofficial
Error downloading https://www.discogs.com/artist/4781-Fellowship?type=Unofficial
Error downloading https://www.discogs.com/artist/58881-Brando?type=Unofficial
Error downloading https://www.discogs.com/artist/6129681-Madonna-Vs-Offer-Nissim?type=Unofficial
Error downloading https://www.discogs.com/artist/688281-Grouper?type=Unofficial
Error downloading https://www.discogs.com/artist/695481-Dingle?type=Unofficial
Error downloading https://www.discogs.com/artist/6981-Gez-Varley?type=Unofficial
Error downloading https://www.discogs.com/artist/80881-KHZ?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/81/unofficial/3664381.p (force=False)
  --> This file is 15.6kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7733 previous data for ModVal=82
Found 334 artists to get
Error downloading https://www.discogs.com/artist/1085582-Bitter-Funeral-Beer-Band?type=Unofficial
Error downloading https://www.discogs.com/artist/364982-Cliff-Burton?type=Unofficial
=========================   discogs   =========================
  ===> Found 7728 previous data for ModVal=83
Found 337 artists to get
Error downloading https://www.discogs.com/artist/1787083-Dekka-Danse?type=Unofficial
Error downloading https://www.discogs.com/artist/221683-Nitty-2?type=Unofficial
Error downloading https://www.discogs.com/artist/390583-The-Gardenias?type=Unofficial
Error downloading https://www.discogs.com/artist/40083-The-Badman?type=Unofficial
Error downloading https://www.discogs.com/artist/4683983-Hot-Legs2?type=Unofficial
Error downloading https://www.discogs.com/artist/4898783-Wayne-Horvitz-Zony-Mash?type=Unofficial
Error downloading https://www.discogs.com/artist/7683-The-Ultimate-Seduction?type=Unofficial
=========================   discogs   =========================
  ===> Found 7721 previous data for ModVal=84
Found 366 artists to get
Error downloading https://www.discogs.com/artist/1079484-Franchise-3?type=Unofficial
Error downloading https://www.discogs.com/artist/1222784-Ritual-Device?type=Unofficial
Error downloading https://www.discogs.com/artist/157284-Axis-5?type=Unofficial
Error downloading https://www.discogs.com/artist/203684-Ras-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/316984-Funk-Doc?type=Unofficial
Error downloading https://www.discogs.com/artist/455284-June-Carter?type=Unofficial
Error downloading https://www.discogs.com/artist/51084-Pamela-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/775484-Jim-Ladd?type=Unofficial
Error downloading https://www.discogs.com/artist/801284-Sam-Sparro?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/84/unofficial/789284.p (force=False)
  --> This file is 18.9kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7612 previous data for ModVal=85
Found 323 artists to get
Error downloading https://www.discogs.com/artist/1576685-NDraman-Blintch?type=Unofficial
Error downloading https://www.discogs.com/artist/1987285-Dispensing-Of-False-Halos?type=Unofficial
Error downloading https://www.discogs.com/artist/253885-T-Lee?type=Unofficial
Error downloading https://www.discogs.com/artist/285-Fierce?type=Unofficial
Error downloading https://www.discogs.com/artist/3855185-DJ-Famous?type=Unofficial
Error downloading https://www.discogs.com/artist/597585-Tommy-Brown-2?type=Unofficial
Error downloading https://www.discogs.com/artist/6038385-DJ-Roshi?type=Unofficial
Error downloading https://www.discogs.com/artist/742285-Jay-Gordon-2?type=Unofficial
Error downloading https://www.discogs.com/artist/90185-Kevin-Gilbert?type=Unofficial
=========================   discogs   =========================
  ===> Found 7830 previous data for ModVal=86
Found 340 artists to get
Error downloading https://www.discogs.com/artist/110086-Trigga?type=Unofficial
Error downloading https://www.discogs.com/artist/1484386-Filipe-Rodrigues?type=Unofficial
Error downloading https://www.discogs.com/artist/1612186-Lionel-Jeffries?type=Unofficial
Error downloading https://www.discogs.com/artist/178686-John-Doe-4?type=Unofficial
Error downloading https://www.discogs.com/artist/1791486-Jonathan-Sielaff?type=Unofficial
Error downloading https://www.discogs.com/artist/1996086-Stay-3?type=Unofficial
Error downloading https://www.discogs.com/artist/2296486-The-Archie-Shepp-Bill-Dixon-Quartet?type=Unofficial
Error downloading https://www.discogs.com/artist/272886-David-Gilden?type=Unofficial
Error downloading https://www.discogs.com/artist/282986-Nikki-Belle?type=Unofficial
Error downloading https://www.discogs.com/artist/335786-David-Friesen?type=Unofficial
Error downloading https://www.discogs.com/artist/405286-King-Hannibal?type=Unofficial
Error downloading https://www.discogs.com/artist/4622386-Invasion-17?type=Unofficial
Error downloading https://www.discogs.com/artist/555486-Shines?type=Unofficial
=========================   discogs   =========================
  ===> Found 7650 previous data for ModVal=87
Found 348 artists to get
Error downloading https://www.discogs.com/artist/232887-Friends?type=Unofficial
Error downloading https://www.discogs.com/artist/4487-Katcha?type=Unofficial
Error downloading https://www.discogs.com/artist/6087-Nexus-21?type=Unofficial
Error downloading https://www.discogs.com/artist/71087-Shaka?type=Unofficial
Error downloading https://www.discogs.com/artist/899287-Dave-Alvin-And-The-Guilty-Men?type=Unofficial
=========================   discogs   =========================
  ===> Found 7628 previous data for ModVal=88
Found 347 artists to get
Error downloading https://www.discogs.com/artist/1135188-Black-Mophia-Clan?type=Unofficial
Error downloading https://www.discogs.com/artist/12888-KK-Null?type=Unofficial
Error downloading https://www.discogs.com/artist/2322588-DJ-Ti%C3%91%C2%91sto?type=Unofficial
Error downloading https://www.discogs.com/artist/2793088-Albacore?type=Unofficial
Error downloading https://www.discogs.com/artist/306688-Martin-Simpson?type=Unofficial
Error downloading https://www.discogs.com/artist/3688-Unreleased-Project?type=Unofficial
Error downloading https://www.discogs.com/artist/49288-Shark-Vegas?type=Unofficial
Error downloading https://www.discogs.com/artist/66388-Blue-Nature?type=Unofficial
Error downloading https://www.discogs.com/artist/98688-Billy-Crystal?type=Unofficial
=========================   discogs   =========================
  ===> Found 7742 previous data for ModVal=89
Found 330 artists to get
Error downloading https://www.discogs.com/artist/1796589-Morticite?type=Unofficial
Error downloading https://www.discogs.com/artist/301589-Artie-Butler?type=Unofficial
Error downloading https://www.discogs.com/artist/334389-Rumbleseat?type=Unofficial
Error downloading https://www.discogs.com/artist/3871389-Skilfingar?type=Unofficial
Error downloading https://www.discogs.com/artist/4230689-Kboy-Jack-Dj-s?type=Unofficial
Error downloading https://www.discogs.com/artist/43589-Quartz-2?type=Unofficial
Error downloading https://www.discogs.com/artist/438789-Arrington-De-Dionyso?type=Unofficial
Error downloading https://www.discogs.com/artist/99289-Amit?type=Unofficial
=========================   discogs   =========================
  ===> Found 7528 previous data for ModVal=90
Found 337 artists to get
Error downloading https://www.discogs.com/artist/11490-Industry-Standard-2?type=Unofficial
Error downloading https://www.discogs.com/artist/136990-Aki-Onda?type=Unofficial
Error downloading https://www.discogs.com/artist/1475990-The-John-Berberian-Ensemble?type=Unofficial
Error downloading https://www.discogs.com/artist/171290-Randys-All-Stars?type=Unofficial
Error downloading https://www.discogs.com/artist/2158190-333?type=Unofficial
Error downloading https://www.discogs.com/artist/225490-DJ-Deep-2?type=Unofficial
Error downloading https://www.discogs.com/artist/300790-Brian-Protheroe?type=Unofficial
Error downloading https://www.discogs.com/artist/31590-Bardo-Pond?type=Unofficial
Error downloading https://www.discogs.com/artist/4573790-Suhaily-Shamsuddin?type=Unofficial
=========================   discogs   =========================
  ===> Found 7609 previous data for ModVal=91
Found 340 artists to get
Error downloading https://www.discogs.com/artist/135991-Joe-Coleman?type=Unofficial
Error downloading https://www.discogs.com/artist/1617791-Siam-Shade?type=Unofficial
Error downloading https://www.discogs.com/artist/1629791-Sir-Collins-The-Black-Diamonds?type=Unofficial
Error downloading https://www.discogs.com/artist/17091-Willy-Washington?type=Unofficial
Error downloading https://www.discogs.com/artist/1716591-Les-Prouters?type=Unofficial
Error downloading https://www.discogs.com/artist/191-David-Alvarado?type=Unofficial
Error downloading https://www.discogs.com/artist/2291-Cause-4-Concern?type=Unofficial
Error downloading https://www.discogs.com/artist/38191-Lost?type=Unofficial
Error downloading https://www.discogs.com/artist/405591-Haley-Gibby?type=Unofficial
Error downloading https://www.discogs.com/artist/5891-Pole-Folder-CP?type=Unofficial
Error downloading https://www.discogs.com/artist/624191-Rik-Emmett?type=Unofficial
Error downloading https://www.discogs.com/artist/97191-The-Eternal-Afflict?type=Unofficial
Error downloading https://www.discogs.com/artist/988591-Ted-Joans?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/91/unofficial/543391.p (force=False)
  --> This file is 17.5kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7805 previous data for ModVal=92
Found 320 artists to get
Error downloading https://www.discogs.com/artist/1206992-Dub-Central?type=Unofficial
Error downloading https://www.discogs.com/artist/132592-Phace?type=Unofficial
Error downloading https://www.discogs.com/artist/189392-Orange-Peel?type=Unofficial
Error downloading https://www.discogs.com/artist/194092-Breathe-3?type=Unofficial
Error downloading https://www.discogs.com/artist/212092-De-Kift?type=Unofficial
Error downloading https://www.discogs.com/artist/42192-Mandr%C3%A9?type=Unofficial
Saving /Volumes/Piggy/Discog/artists-discogs/92/unofficial/251092.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
Saving /Volumes/Piggy/Discog/artists-discogs/92/unofficial/25692.p (force=False)
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
=========================   discogs   =========================
  ===> Found 7627 previous data for ModVal=93
Found 324 artists to get
Error downloading https://www.discogs.com/artist/1013493-Cahill-2?type=Unofficial
Error downloading https://www.discogs.com/artist/216993-Alien-Levi?type=Unofficial
Error downloading https://www.discogs.com/artist/307393-Clarence-Williams?type=Unofficial
Error downloading https://www.discogs.com/artist/385493-The-Quests?type=Unofficial
Error downloading https://www.discogs.com/artist/4167493-Billy-Parker-6?type=Unofficial
Error downloading https://www.discogs.com/artist/450693-Chuck-Berry-His-Combo?type=Unofficial
Error downloading https://www.discogs.com/artist/458093-Fuckpony?type=Unofficial
Error downloading https://www.discogs.com/artist/672493-Fito-P%C3%A1ez?type=Unofficial
Error downloading https://www.discogs.com/artist/993293-Street-Kids?type=Unofficial
=========================   discogs   =========================
  ===> Found 7615 previous data for ModVal=94
Found 351 artists to get
Error downloading https://www.discogs.com/artist/123194-Beth-Gibbons-Rustin-Man?type=Unofficial
Error downloading https://www.discogs.com/artist/21194-DJ-6666-feat-The-Illegals?type=Unofficial
Error downloading https://www.discogs.com/artist/3978394-Maelstrom-26?type=Unofficial
Error downloading https://www.discogs.com/artist/510394-Shinki-Chen?type=Unofficial
Error downloading https://www.discogs.com/artist/5155994-DJB-vs-Who-Da-Fck?type=Unofficial
Error downloading https://www.discogs.com/artist/56194-David-Grubbs?type=Unofficial
=========================   discogs   =========================
  ===> Found 7680 previous data for ModVal=95
Found 319 artists to get
Error downloading https://www.discogs.com/artist/1424095-Isaak?type=Unofficial
Error downloading https://www.discogs.com/artist/259295-DJ-Flapjack?type=Unofficial
Error downloading https://www.discogs.com/artist/29895-Joe-Gibbs-The-Professionals?type=Unofficial
Error downloading https://www.discogs.com/artist/33995-Ed-Motta?type=Unofficial
Error downloading https://www.discogs.com/artist/483695-Boozoo-Chavis?type=Unofficial
Error downloading https://www.discogs.com/artist/494995-Lauren-Newton?type=Unofficial
Error downloading https://www.discogs.com/artist/499495-Brainwork?type=Unofficial
Error downloading https://www.discogs.com/artist/821995-George-Fleming?type=Unofficial
Error downloading https://www.discogs.com/artist/2619095-Jack-Jones-9?type=Unofficial
=========================   discogs   =========================
  ===> Found 7557 previous data for ModVal=96
Found 372 artists to get
Error downloading https://www.discogs.com/artist/1016096-Jeremy-Jay?type=Unofficial
Error downloading https://www.discogs.com/artist/110696-Mummy-Calls?type=Unofficial
Error downloading https://www.discogs.com/artist/124396-DJ-Extreme?type=Unofficial
Error downloading https://www.discogs.com/artist/2131996-KDC?type=Unofficial
Error downloading https://www.discogs.com/artist/255596-Mystic-Tide?type=Unofficial
Error downloading https://www.discogs.com/artist/28596-Nero?type=Unofficial
Error downloading https://www.discogs.com/artist/3396496-Temple-Of-Oblivion?type=Unofficial
Error downloading https://www.discogs.com/artist/3628996-The-Present-5?type=Unofficial
Error downloading https://www.discogs.com/artist/3696-Nucleus-Paradox?type=Unofficial
Error downloading https://www.discogs.com/artist/83696-Frighty?type=Unofficial
Error downloading https://www.discogs.com/artist/840496-The-Dub-Beatles?type=Unofficial
Error downloading https://www.discogs.com/artist/8996-TD?type=Unofficial
=========================   discogs   =========================
  ===> Found 7619 previous data for ModVal=97
Found 322 artists to get
Error downloading https://www.discogs.com/artist/2190897-Konwent?type=Unofficial
Error downloading https://www.discogs.com/artist/2625697-J-Bigga?type=Unofficial
Error downloading https://www.discogs.com/artist/268097-Billy-Nicholls?type=Unofficial
Error downloading https://www.discogs.com/artist/30897-Timex-Social-Club?type=Unofficial
Error downloading https://www.discogs.com/artist/479897-Sonny-Moore?type=Unofficial
=========================   discogs   =========================
  ===> Found 7563 previous data for ModVal=98
Found 321 artists to get
Error downloading https://www.discogs.com/artist/1747298-Dilermando-Reis?type=Unofficial
Error downloading https://www.discogs.com/artist/220598-Q-ic?type=Unofficial
Error downloading https://www.discogs.com/artist/22898-Tool?type=Unofficial
=========================   discogs   =========================
  ===> Found 7646 previous data for ModVal=99
Found 325 artists to get
Error downloading https://www.discogs.com/artist/10199-Diesel?type=Unofficial
Error downloading https://www.discogs.com/artist/123599-Ram-Jam-World?type=Unofficial
Error downloading https://www.discogs.com/artist/19099-Necro?type=Unofficial
Error downloading https://www.discogs.com/artist/193299-Deon-Estus?type=Unofficial
Error downloading https://www.discogs.com/artist/21699-3-Way?type=Unofficial
Error downloading https://www.discogs.com/artist/2430899-Darren-Korb?type=Unofficial
Error downloading https://www.discogs.com/artist/299-Rufige-Kru?type=Unofficial
Error downloading https://www.discogs.com/artist/329799-Zbigniew-Seifert?type=Unofficial
Error downloading https://www.discogs.com/artist/4987899-Cross-Quinn?type=Unofficial
Error downloading https://www.discogs.com/artist/5217299-Ural-Thomas-And-The-Pain?type=Unofficial
Error downloading https://www.discogs.com/artist/87799-Severo-Lombardoni?type=Unofficial"""

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

dbAD = dbArtistsDiscogs()
dbAP = dbArtistsPrimary(dbAD)

removes = []
for i,line in enumerate(vals.split("\n")):
    if i < 2:
        continue
    if not line.startswith("Error downloading "):
        continue
    url = line.replace("Error downloading ", "")
    
    print(url)
    
    url = url.replace("?type=Unofficial", "")
    artistID = dbAP.dutils.getArtistID(url, debug=False)
    savename = dbAP.dutils.getArtistSavename(artistID)
    print(url)

    try:
        dbAP.dutils.downloadArtistURL(url, savename, force=True, sleeptime=2)
    except:
        removes.append(line)
        

In [ ]:
dbAP.dutils.getArtistID("https://www.discogs.com/artist/718900-G-Punkt-2")

In [ ]:

artistURL = "https://www.discogs.com/artist/2428401-Horse-Head-2?type=Unofficial"
print(artistID,savename)